In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#  Change Master, Glossary_content, Amagi, Dayparts

In [3]:
#Change Master And Month****************************************************************************
Channel= 'Towie'
# 'West' or 'East' (ElRey, Towie)
Timezone='East'
# '_ET' for Eeastern or '' for Pacific
ET='_ET'
Day='28'
EOM= '2022-02-28'
Month='February'
PMonth='January'
Year='_2022'
Qn='Q4'
QuarterN=2

Master = pd.read_excel('Master '+Channel+' - '+ Month + Year + '.xlsx', sheet_name='Master')
glossary_content = pd.read_excel('Master '+ Channel +' - '+ Month + Year + '.xlsx', sheet_name= 'Glossary')
glossary_channel =pd.read_excel('Glossary.xlsx', sheet_name='Platform_Channel')


In [4]:
if QuarterN == 1:
    Months=('Oct','Nov','Dec','Jan')
    m1=10
    m2=11
    m3=12
    m4=1
    m31='12'
    m4l='1'
    
elif QuarterN ==2:
    Months=('Oct','Nov','Dec','Jan','Feb')
    m1=10
    m2=11
    m3=12
    m4=1
    m5=2
    m41='1'
    m5l='2'
    
else:
    Months=('Oct','Nov','Dec','Jan','Feb','Mar')
    m1=10
    m2=11
    m3=12
    m4=1
    m5=2
    m6=3
    m51='2'
    m6l='3'

In [5]:
if QuarterN == 1:
    Amagi_dp1=pd.read_csv('October_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp2=pd.read_csv('November_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp3=pd.read_csv('December_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp4=pd.read_csv('January_2022_hourly_noContent'+ET+'.csv')

    Amagi=Amagi_dp4

elif QuarterN == 2:
    Amagi_dp1=pd.read_csv('October_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp2=pd.read_csv('November_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp3=pd.read_csv('December_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp4=pd.read_csv('January_2022_hourly_noContent'+ET+'.csv')
    Amagi_dp5=pd.read_csv('February_2022_hourly_noContent'+ET+'.csv')
    
    Amagi=Amagi_dp5

else:
    Amagi_dp1=pd.read_csv('October_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp2=pd.read_csv('November_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp3=pd.read_csv('December_2021_hourly_noContent'+ET+'.csv')
    Amagi_dp4=pd.read_csv('January_2022_hourly_noContent'+ET+'.csv')
    Amagi_dp5=pd.read_csv('Febuary_2022_hourly_noContent'+ET+'.csv')
    Amagi_dp6=pd.read_csv('March'+Day+'_2022_hourly_noContent'+ET+'.csv')
    
    
    Amagi=Amagi_dp6

In [6]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

In [7]:
Master= Master.loc[:,('HOUR', 'ASSET ID')]
Master.rename(columns= {'ASSET ID': 'Content ID'},inplace=True)

In [8]:
glossary_content.rename(columns= {'ID': 'Content ID'}, inplace=True)
#glossary_content = glossary_content.loc[:,('Content ID', 'TITLE', 'Series/Film','Type')]
glossary_content = glossary_content.loc[:,('Content ID', 'TITLE', 'Series/Film')]

# Merge Master with glossary

In [9]:
Master = pd.merge(Master, glossary_content, on = 'Content ID', how='left')

In [10]:
Master.rename(columns= {'TITLE':'Title'}, inplace=True)

# Datetime Adjustments

In [11]:
Master['TS'] = pd.to_datetime(Master['HOUR'], format = '%Y-%m-%d %H:%M:%S:%f')
Master= Master[['TS', 'Content ID', 'Title', 'Series/Film']]

In [12]:
Master['Hour_Start']= Master['TS'].dt.hour
Master['Minutes_Start']=Master['TS'].dt.minute

In [13]:
Master['Length']= Master['TS'].shift(-1) - Master['TS']
Master['TS End']= Master['TS'].shift(-1)
Master['Hour_End']= Master['TS End'].dt.hour
Master['Minutes_End']= Master['TS End'].dt.minute


In [14]:
#pd.options.display.max_rows
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

# Fixing the last row

In [15]:
Master.iloc[-1, 7]= EOM + ' 23:59:59.27'

In [16]:
Master['TS End']= pd.to_datetime(Master['TS End'], format = '%Y-%m-%d %H:%M:%S')

In [17]:
Master.iloc[-1,6]= Master.iloc[-1,7]- Master.iloc[-1,0]

In [18]:
Master.iloc[-1,8]= 23
Master.iloc[-1,9]= 59

In [19]:
Master['Hour_End']= Master['Hour_End'].astype(int)

# Adding middle hours

In [20]:
#When Lenght crosses 3 hours
Master.loc[Master['Hour_End']== Master['Hour_Start']+2 , 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==0), 'Middle_1_Hour'] = 23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==1), 'Middle_1_Hour'] = 0


#When Legth crosses 4 hours
Master.loc[Master['Hour_End']== Master['Hour_Start']+3, 'Middle_2_Hour'] = Master['Hour_Start']+2
Master.loc[Master['Hour_End']== Master['Hour_Start']+3, 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==0), 'Middle_2_Hour']=23
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==0), 'Middle_1_Hour']=22

Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==1), 'Middle_2_Hour']=0
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==1), 'Middle_1_Hour']=23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==2), 'Middle_2_Hour']=1
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==2), 'Middle_1_Hour']=0


#when Length crosses 5 hours (ex. 3hr15min film starts at 6:50 ends at 10:05, thats 6,7,8,9,10)
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_3_Hour'] = Master['Hour_Start']+3
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_2_Hour'] = Master['Hour_Start']+2
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_3_Hour']=23
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_2_Hour']=22
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_1_Hour']=21

Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_3_Hour']=0
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_2_Hour']=23
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_1_Hour']=22

Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_3_Hour']=1
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_2_Hour']=0
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_1_Hour']=23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_3_Hour']=2
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_2_Hour']=1
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_1_Hour']=0

# Adding the Percent of the total hour

In [21]:
#percentages
#First Hour
Master.loc[Master['Hour_Start']== (Master['Hour_End']),   'First'] = Master['Length']/timedelta(minutes=60)
Master.loc[Master['Hour_Start']+1 == Master['Hour_End'] , 'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+2 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+3 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+4 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60


Master.loc[(Master['Hour_Start']== Master['Hour_End']+23),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+22),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+21),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+20),'First'] = (60 -Master['TS'].dt.minute)/60


#Last Hour
Master.loc[Master['Hour_Start'] != Master['Hour_End'] ,   'Last'] = Master['Minutes_End']/60

#Mid Hours if necessarry
Master.loc[pd.notna(Master['Middle_1_Hour']),'Mid_1']=1
Master.loc[pd.notna(Master['Middle_2_Hour']),'Mid_2']=1
Master.loc[pd.notna(Master['Middle_3_Hour']),'Mid_3']=1



# Merge Amagi with Glossary

In [22]:
Amagi.rename(columns={'Channel Name':'Platform/Channel'}, inplace= True)
Amagi= pd.merge(Amagi, glossary_channel, on= 'Platform/Channel', how='left')

#  Change to desired Channel

In [23]:
#Change Channel Name **************************************************************************************
Amagi=Amagi[Amagi['Channel']== Channel]

In [24]:
Amagi['Date']= pd.to_datetime(Amagi['Date'], format = '%m/%d/%Y %H:%M')

In [25]:
Amagi['Hour']= Amagi['Date'].dt.hour
Amagi['Day']= Amagi['Date'].dt.day

In [26]:
Amagi_Min=Amagi.groupby(['Day','Hour'])['Total Viewership Minutes'].agg(sum).reset_index()
Amagi_Sess=Amagi.groupby(['Day','Hour'])['Session Count'].agg(sum).reset_index()
Amagi_Min.rename(columns={'Total Viewership Minutes':'Min'}, inplace=True)
Amagi_Sess.rename(columns={'Session Count':'Sess'}, inplace=True)


In [27]:
Master['Day']= Master['TS'].dt.day

# Merging Master with Amagi Data

In [28]:
H=('Hour_Start','Middle_1_Hour','Middle_2_Hour','Middle_3_Hour','Hour_End')
M=('start','Mid1','Mid2','Mid3','End')


for i,j in zip(H,M):
    Master= pd.merge(Master, Amagi_Min ,left_on=['Day',i], right_on=['Day', 'Hour'], how='left' )
    Master= pd.merge(Master, Amagi_Sess ,left_on=['Day',i], right_on=['Day', 'Hour'], how='left' )
    Master.drop(columns=['Hour_x','Hour_y'], inplace= True)
    Master.rename(columns={'Min':'Min_Hour_'+j}, inplace=True)
    Master.rename(columns={'Sess':'Sess_Hour_'+j}, inplace= True)

In [29]:
Master.loc[(Master['Hour_Start']== Master['Hour_End']), 'Min_Hour_End']= 0

In [30]:
Master.loc[(Master['Hour_Start']== Master['Hour_End']), 'Sess_Hour_End']= 0

# Fill nan with 0

In [31]:
L=['Last','Mid_1','Mid_2','Mid_3','Min_Hour_Mid1','Min_Hour_Mid2',
 'Min_Hour_Mid3','Min_Hour_End','Sess_Hour_Mid1','Sess_Hour_Mid2',
 'Sess_Hour_Mid3','Sess_Hour_End']
for i in L:
    Master[i]=Master[i].fillna(0)

# Multiply percent of hour with data for whole hour

In [32]:
Master['Total Minutes']= ((Master['First']*Master['Min_Hour_start']) + 
(Master['Mid_1']*Master['Min_Hour_Mid1']) + 
(Master['Mid_2'] * Master['Min_Hour_Mid2']) + 
(Master['Mid_3'] * Master['Min_Hour_Mid3']) +                           
(Master['Last'] * Master['Min_Hour_End']))

Master['Total Session']= ((Master['First'] * Master['Sess_Hour_start']) + 
(Master['Mid_1'] * Master['Sess_Hour_Mid1']) + 
(Master['Mid_2'] * Master['Sess_Hour_Mid2']) + 
(Master['Mid_3'] * Master['Sess_Hour_Mid3']) + 
(Master['Last'] * Master['Sess_Hour_End']))


In [33]:
Master.drop(Master[Master['TS']==Master['TS End']].index, inplace=True)
Master.reset_index()
df= Master.loc[:,('Series/Film','Title','TS','Day','Hour_Start','Minutes_Start', 'Middle_1_Hour','Middle_2_Hour',
                  'Middle_3_Hour','Length','TS End',
                'Hour_End','Minutes_End','Total Minutes','Total Session')]

# Adding Daypart labels

In [34]:
df['Weekday']= df['TS'].dt.weekday

In [35]:
if Timezone == 'West':
    Conditions2 =[
        (df['Weekday'] < 5) & (df['Hour_Start'] < 6) & (df['Hour_Start'] >= 3),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 13) & (df['Hour_Start'] >= 6),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 17) & (df['Hour_Start'] >= 13),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 20) & (df['Hour_Start'] >= 17),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 23) & (df['Hour_Start'] >= 20),
        (df['Weekday'] < 5) & ((df['Hour_Start'] < 3) | (df['Hour_Start'] >= 23)),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 7) & (df['Hour_Start'] >= 3),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 17) & (df['Hour_Start'] >= 7),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 20) & (df['Hour_Start'] >= 17),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 23) & (df['Hour_Start'] >= 20),
        (df['Weekday'] >= 5) & ((df['Hour_Start'] < 3) | (df['Hour_Start'] >= 23)),
    ]
else:
    Conditions2 =[
        (df['Weekday'] < 5) & (df['Hour_Start'] < 9) & (df['Hour_Start'] >= 6),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 16) & (df['Hour_Start'] >= 9),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 20) & (df['Hour_Start'] >= 16),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 23) & (df['Hour_Start'] >= 20),
        (df['Weekday'] < 5) & ((df['Hour_Start'] < 2) | (df['Hour_Start'] >= 23)),
        (df['Weekday'] < 5) & (df['Hour_Start'] < 6) & (df['Hour_Start'] >= 2),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 10) & (df['Hour_Start'] >= 6),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 20) & (df['Hour_Start'] >= 10),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 23) & (df['Hour_Start'] >= 20),
        (df['Weekday'] >= 5) & ((df['Hour_Start'] < 2) | (df['Hour_Start'] >= 23)),
        (df['Weekday'] >= 5) & (df['Hour_Start'] < 6) & (df['Hour_Start'] >= 2),
    ]
    

Daypart2 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

df['Dayparts_Start']= np.select(Conditions2, Daypart2, default= np.nan)

In [36]:
if Timezone == 'West':
    Conditions3 =[
        (df['Weekday'] < 5) & (df['Hour_End'] < 6) & (df['Hour_End'] >= 3),
        (df['Weekday'] < 5) & (df['Hour_End'] < 13) & (df['Hour_End'] >= 6),
        (df['Weekday'] < 5) & (df['Hour_End'] < 17) & (df['Hour_End'] >= 13),
        (df['Weekday'] < 5) & (df['Hour_End'] < 20) & (df['Hour_End'] >= 17),
        (df['Weekday'] < 5) & (df['Hour_End'] < 23) & (df['Hour_End'] >= 20),
        (df['Weekday'] < 5) & ((df['Hour_End'] < 3) | (df['Hour_End'] >= 23)),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 7) & (df['Hour_End'] >= 3),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 17) & (df['Hour_End'] >= 7),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 20) & (df['Hour_End'] >= 17),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 23) & (df['Hour_End'] >= 20),
        (df['Weekday'] >= 5) & ((df['Hour_End'] < 3) | (df['Hour_End'] >= 23)),
    
    ]
else:
    Conditions3 =[
        (df['Weekday'] < 5) & (df['Hour_End'] < 9) & (df['Hour_End'] >= 6),
        (df['Weekday'] < 5) & (df['Hour_End'] < 16) & (df['Hour_End'] >= 9),
        (df['Weekday'] < 5) & (df['Hour_End'] < 20) & (df['Hour_End'] >= 16),
        (df['Weekday'] < 5) & (df['Hour_End'] < 23) & (df['Hour_End'] >= 20),
        (df['Weekday'] < 5) & ((df['Hour_End'] < 2) | (df['Hour_End'] >= 23)),
        (df['Weekday'] < 5) & (df['Hour_End'] < 6) & (df['Hour_End'] >= 2),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 10) & (df['Hour_End'] >= 6),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 20) & (df['Hour_End'] >= 10),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 23) & (df['Hour_End'] >= 20),
        (df['Weekday'] >= 5) & ((df['Hour_End'] < 2) | (df['Hour_End'] >= 23)),
        (df['Weekday'] >= 5) & (df['Hour_End'] < 6) & (df['Hour_End'] >= 2),
    
    ]

Daypart3 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

df['Dayparts_End']= np.select(Conditions3, Daypart3, default= np.nan)

In [37]:
df['com']= df['Dayparts_Start']+df['Dayparts_End']

In [38]:
if Timezone == 'West':
    dayp={ 'com':
    ['Weekday MorningWeekday Day' ,'Weekday DayWeekday Fringe','Weekday FringePrime'
    ,'PrimeLate Night','Late NightOvernight','OvernightWeekday Morning'
    ,'OvernightWeekend Morning','Weekend MorningWeekend Day','Weekend DayPrime (wknd)'
    ,'Prime (wknd)Late Night (wknd)','Late Night (wknd)Overnight (wknd)','Overnight (wknd)Weekend Morning'
    ,'Overnight (wknd)Weekday Morning'], 
          'DP_HR_Cross':
    [6,13,17,20,23,3,3,7,17,20,23,3,3]
         }
else:
    dayp={ 'com':
    ['Weekday MorningWeekday Day' ,'Weekday DayWeekday Fringe','Weekday FringePrime'
    ,'PrimeLate Night','Late NightOvernight','OvernightWeekday Morning'
    ,'OvernightWeekend Morning','Weekend MorningWeekend Day','Weekend DayPrime (wknd)'
    ,'Prime (wknd)Late Night (wknd)','Late Night (wknd)Overnight (wknd)','Overnight (wknd)Weekend Morning'
    ,'Overnight (wknd)Weekday Morning'], 
          'DP_HR_Cross':
    [9,16,20,23,2,6,6,10,20,23,2,6,6]
         }
daypp= pd.DataFrame(dayp, columns= ['com','DP_HR_Cross'])

In [39]:
df= pd.merge(df, daypp, on='com', how='left')

In [40]:
df['H']=pd.to_timedelta(df.DP_HR_Cross, unit='h')

In [41]:
df['date']= df['TS'].dt.date 

In [42]:
df['date']= pd.to_datetime(df['date'])

In [43]:
df['DT_Cross']= df['date']+ df['H']

In [44]:
df['DP_ratio']= (df['DT_Cross']- df['TS'])/ df['Length']
df['DP2_ratio']=(df['TS End']- df['DT_Cross'])/ df['Length']

In [45]:
df.loc[df['DP_ratio']<= .1, 'DP_ratio']=0
df.loc[df['DP_ratio']>= .9, 'DP_ratio']=1
df.loc[df['DP_ratio']<= .1, 'DP2_ratio']=1
df.loc[df['DP_ratio']>= .9, 'DP2_ratio']=0

In [46]:
df['DP_ratio']= df['DP_ratio'].fillna(1)
df['DP2_ratio']= df['DP2_ratio'].fillna(0)

In [47]:
df['Min_total']= df['Total Minutes']
df['Sess_total']= df['Total Session']

In [48]:
df['Min_Daypart']= df['DP_ratio'] * df['Min_total']
df['Min_Daypart2']= df['DP2_ratio']* df['Min_total']

df['Sess_Daypart']=df['DP_ratio']* df['Sess_total']
df['Sess_Daypart2']=df['DP2_ratio']* df['Sess_total']

In [49]:
df['Sess_Duration'] = df['Min_Daypart']/ df['Sess_Daypart']
df['Sess_Duration2'] = df['Min_Daypart2']/ df['Sess_Daypart2']

In [50]:
df['Length_minutes']= df['Length']/timedelta(minutes=1)

In [51]:
df['Length_minutes_DP1']= df['Length_minutes']* df['DP_ratio']
df['Length_minutes_DP2']= df['Length_minutes']* df['DP2_ratio']

In [52]:
df['Minutes per min']= df['Min_Daypart']/df['Length_minutes_DP1']
df['Minutes_Second Daypart']= df['Min_Daypart2']/df['Length_minutes_DP2']

df['Sess per min']= df['Sess_Daypart']/df['Length_minutes_DP1']
df['Sess per min_Second Daypart']= df['Sess_Daypart2']/df['Length_minutes_DP2']



# Create Daypart Averages

In [53]:
if QuarterN == 1:
    Amagi_dp4.rename(columns={'Platform/Channel':'Channel Name'}, inplace=True)
elif QuarterN == 2:
    Amagi_dp5.rename(columns={'Platform/Channel':'Channel Name'}, inplace=True)
else:
    Amagi_dp6.rename(columns={'Platform/Channel':'Channel Name'}, inplace=True)

In [54]:
if QuarterN == 1:
    Amagi_dp1['Date']= pd.to_datetime(Amagi_dp1['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp2['Date']= pd.to_datetime(Amagi_dp2['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp3['Date']= pd.to_datetime(Amagi_dp3['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp4['Date']= pd.to_datetime(Amagi_dp4['Date'], format = '%m/%d/%Y %H:%M')
elif QuarterN == 2:
    Amagi_dp1['Date']= pd.to_datetime(Amagi_dp1['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp2['Date']= pd.to_datetime(Amagi_dp2['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp3['Date']= pd.to_datetime(Amagi_dp3['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp4['Date']= pd.to_datetime(Amagi_dp4['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp5['Date']= pd.to_datetime(Amagi_dp5['Date'], format = '%m/%d/%Y %H:%M')
else:
    Amagi_dp1['Date']= pd.to_datetime(Amagi_dp1['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp2['Date']= pd.to_datetime(Amagi_dp2['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp3['Date']= pd.to_datetime(Amagi_dp3['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp4['Date']= pd.to_datetime(Amagi_dp4['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp5['Date']= pd.to_datetime(Amagi_dp5['Date'], format = '%m/%d/%Y %H:%M')
    Amagi_dp6['Date']= pd.to_datetime(Amagi_dp6['Date'], format = '%m/%d/%Y %H:%M')
    
    


In [55]:
if QuarterN == 1:
    Amagi_DP= pd.concat([Amagi_dp1, Amagi_dp2, Amagi_dp3, Amagi_dp4], axis=0)
elif QuarterN == 2:
    Amagi_DP= pd.concat([Amagi_dp1, Amagi_dp2, Amagi_dp3, Amagi_dp4, Amagi_dp5], axis=0)
else:
    Amagi_DP= pd.concat([Amagi_dp1, Amagi_dp2, Amagi_dp3, Amagi_dp4, Amagi_dp5, Amagi_dp6], axis=0)
    
Amagi_DP['Date']= pd.to_datetime(Amagi_DP['Date'], format = '%m/%d/%Y %H:%M')

In [56]:
Amagi_DP.rename(columns={'Channel Name':'Platform/Channel'}, inplace= True)
Amagi_DP= pd.merge(Amagi_DP, glossary_channel, on= 'Platform/Channel', how='left')

#Change Channel Name*********************************************************************************
Amagi_DP=Amagi_DP[Amagi_DP['Channel']== Channel]

In [57]:
Amagi_DP['Weekday']= Amagi_DP['Date'].dt.weekday
Amagi_DP['Hour']= Amagi_DP['Date'].dt.hour
Amagi_DP['Day']= Amagi_DP['Date'].dt.day
Amagi_DP['Month']= Amagi_DP['Date'].dt.month

In [58]:
ADP= Amagi_DP

In [59]:
Amagi_DP

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Platform,Channel,Weekday,Hour,Day,Month
58,2021-10-01 00:00:00,Towie Roku US,2193,3028.52,1764,Roku,Towie,4,0,1,10
119,2021-10-01 01:00:00,Towie Roku US,1588,3421.42,1278,Roku,Towie,4,1,1,10
175,2021-10-01 02:00:00,Towie Roku US,1294,3559.75,1029,Roku,Towie,4,2,1,10
238,2021-10-01 03:00:00,Towie Roku US,1109,3228.58,883,Roku,Towie,4,3,1,10
297,2021-10-01 04:00:00,Towie Roku US,893,2512.98,730,Roku,Towie,4,4,1,10
...,...,...,...,...,...,...,...,...,...,...,...
237109,2022-02-28 21:00:00,Towie Roku US,3494,5389.03,2812,Roku,Towie,0,21,28,2
237175,2022-02-28 22:00:00,TheOnlyWayIsEssex_LG,245,281.38,217,LG,Towie,0,22,28,2
237176,2022-02-28 22:00:00,Towie Roku US,3194,5255.63,2568,Roku,Towie,0,22,28,2
237238,2022-02-28 23:00:00,TheOnlyWayIsEssex_LG,473,682.07,446,LG,Towie,0,23,28,2


In [60]:
if Timezone == 'West':
    Conditions =[
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 6) & (ADP['Hour'] >= 3),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 13) & (ADP['Hour'] >= 6),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 17) & (ADP['Hour'] >= 13),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 20) & (ADP['Hour'] >= 17),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 23) & (ADP['Hour'] >= 20),
        (ADP['Weekday'] < 5) & ((ADP['Hour'] < 3) | (ADP['Hour'] >= 23)),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 7) & (ADP['Hour'] >= 3),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 17) & (ADP['Hour'] >= 7),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 20) & (ADP['Hour'] >= 17),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 23) & (ADP['Hour'] >= 20),
        (ADP['Weekday'] >= 5) & ((ADP['Hour'] < 3) | (ADP['Hour'] >= 23)),
    
    ]
else:
    Conditions =[
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 9) & (ADP['Hour'] >= 6),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 16) & (ADP['Hour'] >= 9),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 20) & (ADP['Hour'] >= 16),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 23) & (ADP['Hour'] >= 20),
        (ADP['Weekday'] < 5) & ((ADP['Hour'] < 2) | (ADP['Hour'] >= 23)),
        (ADP['Weekday'] < 5) & (ADP['Hour'] < 6) & (ADP['Hour'] >= 2),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 10) & (ADP['Hour'] >= 6),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 20) & (ADP['Hour'] >= 10),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 23) & (ADP['Hour'] >= 20),
        (ADP['Weekday'] >= 5) & ((ADP['Hour'] < 2) | (ADP['Hour'] >= 23)),
        (ADP['Weekday'] >= 5) & (ADP['Hour'] < 6) & (ADP['Hour'] >= 2),
    
    ]
    
Daypart = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

ADP['Daypart']= np.select(Conditions, Daypart, default= np.nan)

In [61]:
ADP= ADP.groupby(['Month','Day','Hour','Daypart']).agg(sum).reset_index()

In [62]:
ADP=ADP.groupby(['Month','Daypart']).mean().reset_index()
ADP['Sess_Duration']= ADP['Total Viewership Minutes']/ ADP['Session Count']

In [63]:
ADPM= ADP.loc[:,('Month','Daypart', 'Total Viewership Minutes')]
ADPS= ADP.loc[:,('Month','Daypart', 'Sess_Duration')]

In [64]:
ADPM=ADPM.pivot( index= ['Daypart'], columns= 'Month')
ADPS=ADPS.pivot( index= ['Daypart'], columns= 'Month')


In [65]:
ADPM.columns= ADPM.columns.droplevel(0)
ADPM.columns.name= None
ADPM=ADPM.reset_index()

ADPS.columns= ADPS.columns.droplevel(0)
ADPS.columns.name=None
ADPS = ADPS.reset_index()

In [66]:
ADPM

,Daypart,1,2,10,11,12
0,Late Night,6175.734921,5382.382833,3437.597937,4027.803030,5422.261739
1,Late Night (wknd),6354.669333,5754.022083,3582.498000,4455.867083,5376.563333
2,Overnight,4946.023690,4628.941625,2633.607738,3196.878523,4170.300326
3,Overnight (wknd),5519.243500,5354.099063,3037.517500,3419.013750,4213.006250
4,Prime,7185.865873,5964.879833,3793.213016,4636.191818,6206.979130
5,Prime (wknd),7190.109333,5632.951250,4045.991333,4903.527500,6108.376250
6,Weekday Day,5718.977279,4860.749500,2934.271224,3511.410455,4580.110870
7,Weekday Fringe,6278.759405,5408.652000,3268.902619,3968.105341,5336.124022
8,Weekday Morning,4784.486032,4272.618833,2527.686190,3014.755455,3899.747391
9,Weekend Day,6473.780600,5490.208250,3543.063800,4403.965250,5127.452875


In [67]:
if QuarterN == 1:
    ADPM[m1]=ADPM[m1]/60
    ADPM[m2]=ADPM[m2]/60
    ADPM[m3]=ADPM[m3]/60
    ADPM[m4]=ADPM[m4]/60

    ADPM[Qn]= (ADPM[m1]+ ADPM[m2]+ ADPM[m3])/3
    ADPS[Qn]= (ADPS[m1]+ ADPS[m2]+ ADPS[m3])/3
elif QuarterN == 2:
    ADPM[m1]=ADPM[m1]/60
    ADPM[m2]=ADPM[m2]/60
    ADPM[m3]=ADPM[m3]/60
    ADPM[m4]=ADPM[m4]/60
    ADPM[m5]=ADPM[m5]/60
    
    ADPM[Qn]= (ADPM[m1]+ ADPM[m2]+ ADPM[m3])/3
    ADPS[Qn]= (ADPS[m1]+ ADPS[m2]+ ADPS[m3])/3
else:
    ADPM[m1]=ADPM[m1]/60
    ADPM[m2]=ADPM[m2]/60
    ADPM[m3]=ADPM[m3]/60
    ADPM[m4]=ADPM[m4]/60
    ADPM[m5]=ADPM[m5]/60
    ADPM[m6]=ADPM[m6]/60
    
    ADPM[Qn]= (ADPM[m1]+ ADPM[m2]+ ADPM[m3])/3
    ADPS[Qn]= (ADPS[m1]+ ADPS[m2]+ ADPS[m3])/3



In [68]:
#Will Use Later
Platform = Amagi_DP

# Create Hourly Trends

In [69]:
if Timezone == 'West':
    Conditions =[
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 6) & (Amagi_DP['Hour'] >= 3),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 13) & (Amagi_DP['Hour'] >= 6),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 17) & (Amagi_DP['Hour'] >= 13),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 20) & (Amagi_DP['Hour'] >= 17),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 23) & (Amagi_DP['Hour'] >= 20),
        (Amagi_DP['Weekday'] < 5) & ((Amagi_DP['Hour'] < 3) | (Amagi_DP['Hour'] >= 23)),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 7) & (Amagi_DP['Hour'] >= 3),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 17) & (Amagi_DP['Hour'] >= 7),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 20) & (Amagi_DP['Hour'] >= 17),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 23) & (Amagi_DP['Hour'] >= 20),
        (Amagi_DP['Weekday'] >= 5) & ((Amagi_DP['Hour'] < 3) | (Amagi_DP['Hour'] >= 23)),

    ]
else:
    Conditions =[
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 9) & (Amagi_DP['Hour'] >= 6),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 16) & (Amagi_DP['Hour'] >= 9),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 20) & (Amagi_DP['Hour'] >= 16),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 23) & (Amagi_DP['Hour'] >= 20),
        (Amagi_DP['Weekday'] < 5) & ((Amagi_DP['Hour'] < 2) | (Amagi_DP['Hour'] >= 23)),
        (Amagi_DP['Weekday'] < 5) & (Amagi_DP['Hour'] < 6) & (Amagi_DP['Hour'] >= 2),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 10) & (Amagi_DP['Hour'] >= 6),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 20) & (Amagi_DP['Hour'] >= 10),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 23) & (Amagi_DP['Hour'] >= 20),
        (Amagi_DP['Weekday'] >= 5) & ((Amagi_DP['Hour'] < 2) | (Amagi_DP['Hour'] >= 23)),
        (Amagi_DP['Weekday'] >= 5) & (Amagi_DP['Hour'] < 6) & (Amagi_DP['Hour'] >= 2),

    ]

Daypart = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

Amagi_DP['Daypart']= np.select(Conditions, Daypart, default= np.nan)

In [70]:
Amagi_DP= Amagi_DP.groupby(['Month','Day','Hour','Daypart']).agg(sum).reset_index()

In [71]:
Amagi_DP['Month']=Amagi_DP['Month'].astype('category')
Amagi_DP['Day']=Amagi_DP['Day'].astype('category')
Amagi_DP['Hour']=Amagi_DP['Hour'].astype('category')

In [72]:
Amagi_DP=Amagi_DP.groupby(['Month','Day','Hour','Daypart']).mean().reset_index()
Amagi_DP['Sess_Duration']= Amagi_DP['Total Viewership Minutes']/ Amagi_DP['Session Count']

In [73]:
Amagi_DP= Amagi_DP.loc[:,('Month','Day','Hour','Daypart','Total Viewership Minutes','Sess_Duration')]

In [74]:
Amagi_DP['Total Viewership Minutes per minute']=Amagi_DP['Total Viewership Minutes']/60

In [75]:
Amagi_DP=Amagi_DP[Amagi_DP['Sess_Duration'].notna()]

In [76]:
Conditions=[
(Amagi_DP['Daypart'] == 'Weekday Morning') | (Amagi_DP['Daypart']=='Weekday Day') | (Amagi_DP['Daypart']=='Weekday Fringe') | (Amagi_DP['Daypart']=='Prime') | (Amagi_DP['Daypart']=='Late Night') | (Amagi_DP['Daypart']=='Overnight'),
(Amagi_DP['Daypart']== 'Weekend Morning') | (Amagi_DP['Daypart']=='Weekend Day') | (Amagi_DP['Daypart']=='Prime (wknd)') | (Amagi_DP['Daypart']=='Late Night (wknd)') | (Amagi_DP['Daypart']=='Overnight (wknd)')    
]

results= ['Weekday', 'Weekend']

Amagi_DP['week/weekend']= np.select(Conditions, results, default=np.nan)

# Daypart Averages Table

In [77]:
Amagi_DP_weekdays= Amagi_DP[Amagi_DP['week/weekend']== 'Weekday']
Amagi_DP_weekends= Amagi_DP[Amagi_DP['week/weekend']== 'Weekend']

In [78]:
Amagi_DP_weekdays=Amagi_DP_weekdays.groupby(['Month','Hour','Daypart']).mean().reset_index()
Amagi_DP_weekends=Amagi_DP_weekends.groupby(['Month','Hour','Daypart']).mean().reset_index()

Amagi_DP_weekdays.dropna(inplace=True)
Amagi_DP_weekends.dropna(inplace=True)


In [79]:
ADPw=Amagi_DP_weekdays.pivot(index=['Hour'],columns='Month',values=['Total Viewership Minutes',
                                                                    'Sess_Duration']).swaplevel(0,1,axis=1).reset_index()
ADPwk=Amagi_DP_weekends.pivot(index=['Hour'],columns='Month',values=['Total Viewership Minutes',
                                                                    'Sess_Duration']).swaplevel(0,1,axis=1).reset_index()

In [80]:
ADPw.columns= ADPw.columns.droplevel(1)
ADPwk.columns= ADPwk.columns.droplevel(1)

In [81]:
ADPw.rename(columns={1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec','Hour':'Hour(24-Pac)'},inplace=True)
ADPwk.rename(columns={1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec','Hour':'Hour(24-Pac)'},inplace=True)

In [82]:
Con5=[
    (ADPw['Hour(24-Pac)']== 0),(ADPw['Hour(24-Pac)']== 1),(ADPw['Hour(24-Pac)']== 2),(ADPw['Hour(24-Pac)']== 3),
    (ADPw['Hour(24-Pac)']== 4),(ADPw['Hour(24-Pac)']== 5),(ADPw['Hour(24-Pac)']== 6),(ADPw['Hour(24-Pac)']== 7),
    (ADPw['Hour(24-Pac)']== 8),(ADPw['Hour(24-Pac)']== 9),(ADPw['Hour(24-Pac)']== 10),(ADPw['Hour(24-Pac)']== 11),
    (ADPw['Hour(24-Pac)']== 12),(ADPw['Hour(24-Pac)']== 13),(ADPw['Hour(24-Pac)']== 14),(ADPw['Hour(24-Pac)']== 15),
    (ADPw['Hour(24-Pac)']== 16),(ADPw['Hour(24-Pac)']== 17),(ADPw['Hour(24-Pac)']== 18),(ADPw['Hour(24-Pac)']== 19),
    (ADPw['Hour(24-Pac)']== 20),(ADPw['Hour(24-Pac)']== 21),(ADPw['Hour(24-Pac)']== 22),(ADPw['Hour(24-Pac)']== 23)  
]

Con6=[
    (ADPwk['Hour(24-Pac)']== 0),(ADPwk['Hour(24-Pac)']== 1),(ADPwk['Hour(24-Pac)']== 2),(ADPwk['Hour(24-Pac)']== 3),
    (ADPwk['Hour(24-Pac)']== 4),(ADPwk['Hour(24-Pac)']== 5),(ADPwk['Hour(24-Pac)']== 6),(ADPwk['Hour(24-Pac)']== 7),
    (ADPwk['Hour(24-Pac)']== 8),(ADPwk['Hour(24-Pac)']== 9),(ADPwk['Hour(24-Pac)']== 10),(ADPwk['Hour(24-Pac)']== 11),
    (ADPwk['Hour(24-Pac)']== 12),(ADPwk['Hour(24-Pac)']== 13),(ADPwk['Hour(24-Pac)']== 14),(ADPwk['Hour(24-Pac)']== 15),
    (ADPwk['Hour(24-Pac)']== 16),(ADPwk['Hour(24-Pac)']== 17),(ADPwk['Hour(24-Pac)']== 18),(ADPwk['Hour(24-Pac)']== 19),
    (ADPwk['Hour(24-Pac)']== 20),(ADPwk['Hour(24-Pac)']== 21),(ADPwk['Hour(24-Pac)']== 22),(ADPwk['Hour(24-Pac)']== 23)  
]

if Timezone == 'West':

    pac=['0am-pt','1am-pt','2am-pt','3am-pt','4am-pt','5am-pt',
        '6am-pt','7am-pt','8am-pt','9am-pt','10am-pt','11am-pt',
        '12pm-pt','1pm-pt','2pm-pt','3pm-pt','4pm-pt','5pm-pt',
        '6pm-pt','7pm-pt','8pm-pt','9pm-pt','10pm-pt','11pm-pt']

    east=['3am-et','4am-et','5am-et','6am-et','7am-et','8am-et',
          '9am-et','10am-et','11am-et','12pm-et','1pm-et','2pm-et',
          '3pm-et','4pm-et','5pm-et','6pm-et','7pm-et','8pm-et',
          '9pm-et','10pm-et','11pm-et','0am-et','1am-et','2am-et']
else:
    pac=['9pm-pt','10pm-pt','11pm-pt','0am-pt','1am-pt','2am-pt',
         '3am-pt','4am-pt','5am-pt','6am-pt','7am-pt','8am-pt',
         '9am-pt','10am-pt','11am-pt','12pm-pt','1pm-pt','2pm-pt',
         '3pm-pt','4pm-pt','5pm-pt','6pm-pt','7pm-pt','8pm-pt']

    east=['0am-et','1am-et','2am-et','3am-et','4am-et','5am-et',
          '6am-et','7am-et','8am-et','9am-et','10am-et','11am-et',
          '12pm-et','1pm-et','2pm-et','3pm-et','4pm-et','5pm-et',
          '6pm-et','7pm-et','8pm-et','9pm-et','10pm-et','11pm-et']

ADPw['Pacific_Time']=np.select(Con5,pac,default=np.nan)
ADPw['East_Time']=np.select(Con5,east,default=np.nan)

ADPwk['Pacific_Time']=np.select(Con6,pac,default=np.nan)
ADPwk['East_Time']=np.select(Con6,east,default=np.nan)

In [83]:
ADPwk

Month,Hour(24-Pac),Jan,Feb,Oct,Nov,Dec,Jan,Feb,Oct,Nov,Dec,Pacific_Time,East_Time
0,0,6250.991,5858.14625,3499.588,4318.04750,5452.45125,2.092789,2.479146,1.365010,1.693573,1.926859,9pm-pt,0am-et
1,1,5853.199,5817.52625,3319.772,4469.84250,5013.69375,2.555148,3.080818,1.747212,2.201526,2.385421,10pm-pt,1am-et
2,2,5807.073,5753.38250,3163.076,3693.79625,4618.36750,3.270013,3.666211,2.238166,2.736674,2.805897,11pm-pt,2am-et
3,3,5677.048,5648.55625,3187.302,3515.80375,4288.05750,3.753599,4.382546,2.547005,3.138560,3.114069,0am-pt,3am-et
4,4,5408.546,5177.35375,2971.304,3263.03625,4012.32875,4.309995,4.710424,2.893783,3.214854,3.423715,1am-pt,4am-et
5,5,5184.307,4837.10375,2828.388,3203.41875,3933.27125,4.149579,4.342514,2.849650,2.980906,3.213076,2am-pt,5am-et
6,6,5131.441,4795.87750,2747.645,3287.57375,3991.71625,3.618852,3.924059,2.489915,2.631666,2.766141,3am-pt,6am-et
7,7,5081.700,4623.96625,2802.964,3421.94750,4184.92625,2.982949,3.156550,1.974687,2.113333,2.446484,4am-pt,7am-et
8,8,5232.243,4865.74750,2998.393,3747.62625,4293.57625,2.416382,2.482491,1.596178,1.773002,1.951770,5am-pt,8am-et
9,9,5503.384,5140.38125,3181.408,3993.78625,4400.06875,2.005653,2.135888,1.355983,1.618981,1.714568,6am-pt,9am-et


In [84]:
ADPw['Wk/Wknd']='Weekdays'
ADPwk['Wk/Wknd']='Weekends'

In [85]:
#Renaming Columns, be careful what you name them

if QuarterN == 1:
    ADPw.columns=['Hour(24-Pac)',Months[3],Months[0],Months[1],Months[2],Months[3]+'S',Months[0]+'S',Months[1]+'S',Months[2]+'S','Pacific_Time','East_Time','Wk/Wknd']
    ADPwk.columns=['Hour(24-Pac)',Months[3],Months[0],Months[1],Months[2],Months[3]+'S',Months[0]+'S',Months[1]+'S',Months[2]+'S','Pacific_Time','East_Time','Wk/Wknd']
elif QuarterN == 2:
    ADPw.columns=['Hour(24-Pac)',Months[3],Months[4],Months[0],Months[1],Months[2],Months[3]+'S',Months[4]+'S',Months[0]+'S',Months[1]+'S',Months[2]+'S','Pacific_Time','East_Time','Wk/Wknd']
    ADPwk.columns=['Hour(24-Pac)',Months[3],Months[4],Months[0],Months[1],Months[2],Months[3]+'S',Months[4]+'S',Months[0]+'S',Months[1]+'S',Months[2]+'S','Pacific_Time','East_Time','Wk/Wknd']
else:
    ADPw.columns=['Hour(24-Pac)',Months[3],Months[4],Months[5],Months[0],Months[1],Months[2],Months[3]+'S',Months[4]+'S',Months[5]+'S',Months[0]+'S',Months[1]+'S',Months[2]+'S','Pacific_Time','East_Time','Wk/Wknd']
    ADPwk.columns=['Hour(24-Pac)',Months[3],Months[4],Months[5],Months[0],Months[1],Months[2],Months[3]+'S',Months[4]+'S',Months[5]+'S',Months[0]+'S',Months[1]+'S',Months[2]+'S','Pacific_Time','East_Time','Wk/Wknd']
    
    

In [86]:
ADPw['Hour(24-Pac)']=ADPw['Hour(24-Pac)'].astype(int)
ADPwk['Hour(24-Pac)']=ADPwk['Hour(24-Pac)'].astype(int)

In [87]:
if Timezone == 'West':
    Conditions =[
        (ADPw['Hour(24-Pac)'] < 6) & (ADPw['Hour(24-Pac)'] >= 3),
        (ADPw['Hour(24-Pac)'] < 13) & (ADPw['Hour(24-Pac)'] >= 6),
        (ADPw['Hour(24-Pac)'] < 17) & (ADPw['Hour(24-Pac)'] >= 13),
        (ADPw['Hour(24-Pac)'] < 20) & (ADPw['Hour(24-Pac)'] >= 17),
        (ADPw['Hour(24-Pac)'] < 23) & (ADPw['Hour(24-Pac)'] >= 20),
        ((ADPw['Hour(24-Pac)'] < 3) | (ADPw['Hour(24-Pac)'] >= 23)),
        ]
else:
    Conditions =[
        (ADPw['Hour(24-Pac)'] < 9) & (ADPw['Hour(24-Pac)'] >= 6),
        (ADPw['Hour(24-Pac)'] < 16) & (ADPw['Hour(24-Pac)'] >= 9),
        (ADPw['Hour(24-Pac)'] < 20) & (ADPw['Hour(24-Pac)'] >= 16),
        (ADPw['Hour(24-Pac)'] < 23) & (ADPw['Hour(24-Pac)'] >= 20),
        ((ADPw['Hour(24-Pac)'] < 2) | (ADPw['Hour(24-Pac)'] >= 23)),
        (ADPw['Hour(24-Pac)'] < 6) & (ADPw['Hour(24-Pac)'] >= 2),
        ]
    
Daypart = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
         ]
ADPw['Daypart']= np.select(Conditions, Daypart, default= np.nan)

In [88]:
if Timezone == 'West':
    Conditions =[
        (ADPwk['Hour(24-Pac)'] < 7) & (ADPwk['Hour(24-Pac)'] >= 3),
        (ADPwk['Hour(24-Pac)'] < 17) & (ADPwk['Hour(24-Pac)'] >= 7),
        (ADPwk['Hour(24-Pac)'] < 20) & (ADPwk['Hour(24-Pac)'] >= 17),
        (ADPwk['Hour(24-Pac)'] < 23) & (ADPwk['Hour(24-Pac)'] >= 20),
        ((ADPwk['Hour(24-Pac)'] < 3) | (ADPwk['Hour(24-Pac)'] >= 23))
        ]
else:
    Conditions =[
        (ADPwk['Hour(24-Pac)'] < 10) & (ADPwk['Hour(24-Pac)'] >= 6),
        (ADPwk['Hour(24-Pac)'] < 20) & (ADPwk['Hour(24-Pac)'] >= 10),
        (ADPwk['Hour(24-Pac)'] < 23) & (ADPwk['Hour(24-Pac)'] >= 20),
        ((ADPwk['Hour(24-Pac)'] < 2) | (ADPwk['Hour(24-Pac)'] >= 23)),
        (ADPwk['Hour(24-Pac)'] < 6) & (ADPwk['Hour(24-Pac)'] >= 2)
        ]
    
Daypart = ['Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]
ADPwk['Daypart']= np.select(Conditions, Daypart, default= np.nan)

In [89]:
if QuarterN == 1:
    #Minutes
    AMinWk=ADPw.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3])]
    AMinWknd=ADPwk.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3])]

    #Session Duration
    ASessWk=ADPw.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0]+'S',Months[1]+'S',Months[2]+'S',Months[3]+'S')]
    ASessWknd=ADPwk.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0]+'S',Months[1]+'S',Months[2]+'S',Months[3]+'S')]

    #Change name for Session Duration
    ASessWk.columns=['Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3]]
    ASessWknd.columns=['Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3]]
    
elif QuarterN == 2:
    #Minutes
    AMinWk=ADPw.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4])]
    AMinWknd=ADPwk.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4])]

    #Session Duration
    ASessWk=ADPw.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0]+'S',Months[1]+'S',Months[2]+'S',Months[3]+'S',Months[4]+'S')]
    ASessWknd=ADPwk.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0]+'S',Months[1]+'S',Months[2]+'S',Months[3]+'S',Months[4]+'S')]

    #Change name for Session Duration
    ASessWk.columns=['Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4]]
    ASessWknd.columns=['Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4]]
    
else:
    #Minutes
    AMinWk=ADPw.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5])]
    AMinWknd=ADPwk.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5])]

    #Session Duration
    ASessWk=ADPw.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0]+'S',Months[1]+'S',Months[2]+'S',Months[3]+'S',Months[4]+'S',Months[5]+'S')]
    ASessWknd=ADPwk.loc[:,('Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0]+'S',Months[1]+'S',Months[2]+'S',Months[3]+'S',Months[4]+'S',Months[5]+'S')]

    #Change name for Session Duration
    ASessWk.columns=['Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5]]
    ASessWknd.columns=['Wk/Wknd','Pacific_Time','Hour(24-Pac)','East_Time','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5]]
    

In [90]:
AMinWk[Qn]=(AMinWk[Months[0]]+AMinWk[Months[1]]+AMinWk[Months[2]])/3
ASessWk[Qn]=(ASessWk[Months[0]]+ASessWk[Months[1]]+ASessWk[Months[2]])/3
AMinWknd[Qn]=(AMinWknd[Months[0]]+AMinWknd[Months[1]]+AMinWknd[Months[2]])/3
ASessWknd[Qn]=(ASessWknd[Months[0]]+ASessWknd[Months[1]]+ASessWknd[Months[2]])/3


In [91]:
MonthlyAVGs=pd.concat([AMinWk,ASessWk,AMinWknd,ASessWknd], axis=1)

# Daypart Summ

In [92]:
DPWM=AMinWk
DPWS=ASessWk
DPWkM=AMinWknd
DPWkS=ASessWknd

In [93]:
DPWM=DPWM.groupby('Daypart').mean().reset_index()
DPWS=DPWS.groupby('Daypart').mean().reset_index()
DPWkM=DPWkM.groupby('Daypart').mean().reset_index()
DPWkS=DPWkS.groupby('Daypart').mean().reset_index()



In [94]:
DPWM['Metric']='Minutes'
DPWS['Metric']='Session Duration'
DPWkM['Metric']='Minutes'
DPWkS['Metric']='Session Duration'
    
DPWM['Wk/Wknd']='Weekday'
DPWS['Wk/Wknd']='Weekday'
DPWkM['Wk/Wknd']='Weekend'
DPWkS['Wk/Wknd']='Weekend'

In [95]:
DPWM

,Daypart,Hour(24-Pac),Oct,Nov,Dec,Jan,Feb,Q4,Metric,Wk/Wknd
0,Late Night,8.0,3437.597937,4027.803030,5422.261739,6175.734921,5382.382833,4295.887569,Minutes,Weekday
1,Overnight,3.5,2633.607738,3196.878523,4170.300326,4946.023690,4628.941625,3333.595529,Minutes,Weekday
2,Prime,21.0,3793.213016,4636.191818,6206.979130,7185.865873,5964.879833,4878.794655,Minutes,Weekday
3,Weekday Day,12.0,2934.271224,3511.410455,4580.110870,5718.977279,4860.749500,3675.264183,Minutes,Weekday
4,Weekday Fringe,17.5,3268.902619,3968.105341,5336.124022,6278.759405,5408.652000,4191.043994,Minutes,Weekday
5,Weekday Morning,7.0,2527.686190,3014.755455,3899.747391,4784.486032,4272.618833,3147.396345,Minutes,Weekday


In [96]:
if QuarterN == 1:
    DPWM=DPWM.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Qn)]
    DPWkM=DPWkM.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Qn)]

    DPWS=DPWS.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Qn)]
    DPWkS=DPWkS.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Qn)]
elif QuarterN == 2:
    DPWM=DPWM.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Qn)]
    DPWkM=DPWkM.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Qn)]

    DPWS=DPWS.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Qn)]
    DPWkS=DPWkS.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Qn)]
else:
    DPWM=DPWM.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5],Qn)]
    DPWkM=DPWkM.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5],Qn)]

    DPWS=DPWS.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5],Qn)]
    DPWkS=DPWkS.loc[:,('Metric','Wk/Wknd','Daypart',Months[0],Months[1],Months[2],Months[3],Months[4],Months[5],Qn)]

In [97]:
DPs=DPWM.append([DPWkM,DPWS,DPWkS])

In [98]:
if QuarterN == 1:
    DPs[Months[3]+' V '+Months[2]]= (DPs[Months[3]]-DPs[Months[2]])/DPs[Months[2]] 
    DPs[Months[3]+' V '+Months[1]]= (DPs[Months[3]]-DPs[Months[1]])/DPs[Months[1]] 
    DPs[Months[3]+' V '+Qn]       = (DPs[Months[3]]-DPs[Qn])/DPs[Qn] 
elif QuarterN ==2:
    DPs[Months[4]+' V '+Months[3]]= (DPs[Months[4]]-DPs[Months[3]])/DPs[Months[3]] 
    DPs[Months[4]+' V '+Months[2]]= (DPs[Months[4]]-DPs[Months[2]])/DPs[Months[2]] 
    DPs[Months[4]+' V '+Qn]       = (DPs[Months[4]]-DPs[Qn])/DPs[Qn] 
else:
    DPs[Months[5]+' V '+Months[4]]= (DPs[Months[5]]-DPs[Months[4]])/DPs[Months[4]] 
    DPs[Months[5]+' V '+Months[3]]= (DPs[Months[5]]-DPs[Months[3]])/DPs[Months[3]] 
    DPs[Months[5]+' V '+Qn]       = (DPs[Months[5]]-DPs[Qn])/DPs[Qn] 

In [99]:
DPs

,Metric,Wk/Wknd,Daypart,Oct,Nov,Dec,Jan,Feb,Q4,Feb V Jan,Feb V Dec,Feb V Q4
0,Minutes,Weekday,Late Night,3437.597937,4027.803030,5422.261739,6175.734921,5382.382833,4295.887569,-0.128463,-0.007355,0.252915
1,Minutes,Weekday,Overnight,2633.607738,3196.878523,4170.300326,4946.023690,4628.941625,3333.595529,-0.064108,0.109978,0.388573
2,Minutes,Weekday,Prime,3793.213016,4636.191818,6206.979130,7185.865873,5964.879833,4878.794655,-0.169915,-0.039004,0.222613
3,Minutes,Weekday,Weekday Day,2934.271224,3511.410455,4580.110870,5718.977279,4860.749500,3675.264183,-0.150067,0.061273,0.322558
4,Minutes,Weekday,Weekday Fringe,3268.902619,3968.105341,5336.124022,6278.759405,5408.652000,4191.043994,-0.138580,0.013592,0.290526
5,Minutes,Weekday,Weekday Morning,2527.686190,3014.755455,3899.747391,4784.486032,4272.618833,3147.396345,-0.106985,0.095614,0.357509
0,Minutes,Weekend,Late Night (wknd),3582.498000,4455.867083,5376.563333,6354.669333,5754.022083,4471.642806,-0.094521,0.070204,0.286780
1,Minutes,Weekend,Overnight (wknd),3037.517500,3419.013750,4213.006250,5519.243500,5354.099062,3556.512500,-0.029922,0.270850,0.505435
2,Minutes,Weekend,Prime (wknd),4045.991333,4903.527500,6108.376250,7190.109333,5632.951250,5019.298361,-0.216569,-0.077832,0.122259
3,Minutes,Weekend,Weekend Day,3543.063800,4403.965250,5127.452875,6473.780600,5490.208250,4358.160642,-0.151932,0.070748,0.259754


# Merge Daypart Averages df

In [100]:
dfmin=pd.merge(df,ADPM, left_on='Dayparts_Start', right_on='Daypart', how='left')
dfmin=pd.merge(dfmin,ADPM, left_on='Dayparts_End', right_on='Daypart', how='left')

In [101]:
dfsessD=pd.merge(df,ADPS, left_on='Dayparts_Start', right_on='Daypart', how='left')
dfsessD=pd.merge(dfsessD,ADPS, left_on='Dayparts_End', right_on='Daypart', how='left')

In [102]:
if QuarterN == 1:
    dfmin[ Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[Qn+'_x'])/dfmin[Qn+'_x']
    dfmin[ Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[m31+'_x'])/dfmin[m31+'_x']
    dfmin[ Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[m4l+'_x'])/dfmin[m4l+'_x']


    dfmin[ Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[Qn+'_y'])/dfmin[Qn+'_y']
    dfmin[ Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[m31+'_y'])/dfmin[m31+'_y']
    dfmin[ Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[m4l+'_y'])/dfmin[m4l+'_y']
    
elif QuarterN == 2:
    dfmin[ Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[Qn+'_x'])/dfmin[Qn+'_x']
    dfmin[ Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[m41+'_x'])/dfmin[m41+'_x']
    dfmin[ Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[m5l+'_x'])/dfmin[m5l+'_x']


    dfmin[ Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[Qn+'_y'])/dfmin[Qn+'_y']
    dfmin[ Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[m41+'_y'])/dfmin[m41+'_y']
    dfmin[ Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[m5l+'_y'])/dfmin[m5l+'_y']
else:
    dfmin[ Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[Qn+'_x'])/dfmin[Qn+'_x']
    dfmin[ Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[m51+'_x'])/dfmin[m51+'_x']
    dfmin[ Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)']= (dfmin['Minutes per min']- dfmin[m6l+'_x'])/dfmin[m6l+'_x']


    dfmin[ Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[Qn+'_y'])/dfmin[Qn+'_y']
    dfmin[ Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[m51+'_y'])/dfmin[m51+'_y']
    dfmin[ Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart']= (dfmin['Minutes_Second Daypart']- dfmin[m6l+'_y'])/dfmin[m6l+'_y']

In [103]:
if QuarterN == 1:
    dfsessD[ Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[Qn+'_x'])/dfsessD[Qn+'_x']
    dfsessD[ Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[m31+'_x'])/dfsessD[m31+'_x']
    dfsessD[ Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[m4l+'_x'])/dfsessD[m4l+'_x']


    dfsessD[ Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[Qn+'_y'])/dfsessD[Qn+'_y']
    dfsessD[ Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[m31+'_y'])/dfsessD[m31+'_y']
    dfsessD[ Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[m4l+'_y'])/dfsessD[m4l+'_y']
elif QuarterN ==2:
    dfsessD[ Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[Qn+'_x'])/dfsessD[Qn+'_x']
    dfsessD[ Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[m41+'_x'])/dfsessD[m41+'_x']
    dfsessD[ Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[m5l+'_x'])/dfsessD[m5l+'_x']


    dfsessD[ Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[Qn+'_y'])/dfsessD[Qn+'_y']
    dfsessD[ Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[m41+'_y'])/dfsessD[m41+'_y']
    dfsessD[ Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[m5l+'_y'])/dfsessD[m5l+'_y']
else:
    dfsessD[ Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[Qn+'_x'])/dfsessD[Qn+'_x']
    dfsessD[ Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[m51+'_x'])/dfsessD[m51+'_x']
    dfsessD[ Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)']= (dfsessD['Sess_Duration']- dfsessD[m6l+'_x'])/dfsessD[m6l+'_x']


    dfsessD[ Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[Qn+'_y'])/dfsessD[Qn+'_y']
    dfsessD[ Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[m51+'_y'])/dfsessD[m51+'_y']
    dfsessD[ Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart']= (dfsessD['Sess_Duration2']- dfsessD[m6l+'_y'])/dfsessD[m6l+'_y']
    

In [104]:
dfmin['W']=dfmin['TS'].dt.weekday
dfsessD['W']=dfsessD['TS'].dt.weekday

In [105]:
dfminsess=[dfmin,dfsessD]

for i in dfminsess:
    Condweek=[
        i['W']== 0,i['W']== 1,i['W']== 2,i['W']== 3,
        i['W']== 4,i['W']== 5,i['W']== 6   
    ]
    weekdays= ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
    
    i['Weekday']= np.select(Condweek,weekdays, default=np.nan)

In [106]:
dfmin['Minutes per Minute']= dfmin['Minutes per min'].fillna(dfmin['Minutes_Second Daypart'])
dfsessD['Session Duration']= dfsessD['Sess_Duration'].fillna(dfsessD['Sess_Duration2'])

In [107]:
if QuarterN == 1:
    dfmin=dfmin.loc[:,('TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                       Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                       Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                       Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                       'Dayparts_End',
                       Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                       Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart',
                       Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                       'Minutes per Minute')]   
elif QuarterN == 2:
    dfmin=dfmin.loc[:,('TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                       Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                       Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                       Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                       'Dayparts_End',
                       Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                       Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                       Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                       'Minutes per Minute')]   
else:
    dfmin=dfmin.loc[:,('TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                       Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                       Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                       Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)',
                       'Dayparts_End',
                       Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                       Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                       Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart',
                       'Minutes per Minute')]   

In [108]:
if QuarterN == 1:
    dfsessD=dfsessD.loc[:,('TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                           Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                           Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                           Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                           'Dayparts_End',
                           Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                           Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart',
                           Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                           'Session Duration')]       
elif QuarterN == 2:
    dfsessD=dfsessD.loc[:,('TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                           Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                           Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                           Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                           'Dayparts_End',
                           Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                           Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                           Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                           'Session Duration')]       
else:
    dfsessD=dfsessD.loc[:,('TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                           Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                           Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                           Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)',
                           'Dayparts_End',
                           Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                           Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                           Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart',
                           'Session Duration')]       

# Merging Minutes Schedule with Sess Schedule

In [109]:
Dfmin=dfmin.rename(columns={'Minutes per Minute':'(Minutes per Minute) or (Session Duration)'})
DfsessD=dfsessD.rename(columns={'Session Duration':'(Minutes per Minute) or (Session Duration)'})

Dfmin['Metric']='Minutes Viewed'
DfsessD['Metric']='Session Duration'

Schedule_Data = pd.concat([Dfmin,DfsessD])

if QuarterN == 1:
    Schedule_Data = Schedule_Data.loc[:,('Metric','TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                           Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                           Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                           Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                           'Dayparts_End',
                           Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                           Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart',
                           Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                           '(Minutes per Minute) or (Session Duration)')]
elif QuarterN == 2:
    Schedule_Data = Schedule_Data.loc[:,('Metric','TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                           Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                           Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                           Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                           'Dayparts_End',
                           Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                           Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                           Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                           '(Minutes per Minute) or (Session Duration)')]
else:
    Schedule_Data = Schedule_Data.loc[:,('Metric','TS','Weekday','Series/Film','Length_minutes', 'Title', 'Dayparts_Start',
                           Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                           Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                           Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)',
                           'Dayparts_End',
                           Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                           Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                           Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart',
                           '(Minutes per Minute) or (Session Duration)')]

# Schedule Data Clean up

In [110]:
Schedule_Data['Daypart Start/End']= Schedule_Data['Dayparts_Start']+ '/' +Schedule_Data['Dayparts_End']

if QuarterN == 1:
    Schedule_Data[Months[3]+'(Performance) Vs '+ Qn +'(Daypart avg)']= Schedule_Data.loc[:,(Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)' , Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart')].mean(axis=1)
    Schedule_Data[Months[3]+'(Performance) Vs '+ Months[2] +'(Daypart avg)']= Schedule_Data.loc[:,(Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)' , Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart')].mean(axis=1)
    Schedule_Data[Months[3]+'(Performance) Vs '+ Months[3] +'(Daypart avg)']= Schedule_Data.loc[:,(Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)' , Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart')].mean(axis=1)
elif QuarterN == 2:
    Schedule_Data[Months[4]+'(Performance) Vs '+ Qn +'(Daypart avg)']= Schedule_Data.loc[:,(Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)' , Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart')].mean(axis=1)
    Schedule_Data[Months[4]+'(Performance) Vs '+ Months[3] +'(Daypart avg)']= Schedule_Data.loc[:,(Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)' , Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart')].mean(axis=1)
    Schedule_Data[Months[4]+'(Performance) Vs '+ Months[4] +'(Daypart avg)']= Schedule_Data.loc[:,(Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)' , Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart')].mean(axis=1)
else:
    Schedule_Data[Months[5]+'(Performance) Vs '+ Qn +'(Daypart avg)']= Schedule_Data.loc[:,(Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)' , Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart')].mean(axis=1)
    Schedule_Data[Months[5]+'(Performance) Vs '+ Months[4] +'(Daypart avg)']= Schedule_Data.loc[:,(Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)' , Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart')].mean(axis=1)
    Schedule_Data[Months[5]+'(Performance) Vs '+ Months[5] +'(Daypart avg)']= Schedule_Data.loc[:,(Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)' , Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart')].mean(axis=1)

In [111]:
if QuarterN == 1:
    Schedule_Data = Schedule_Data.loc[:,('Metric','TS','Weekday','Series/Film','Length_minutes', 'Title', 'Daypart Start/End',
                           Months[3]+'(Performance) Vs '+ Qn +'(Daypart avg)',
                           Months[3]+'(Performance) Vs '+ Months[2] +'(Daypart avg)',
                           Months[3]+'(Performance) Vs '+ Months[3] +'(Daypart avg)',
                           '(Minutes per Minute) or (Session Duration)')]
elif QuarterN == 2:
        Schedule_Data = Schedule_Data.loc[:,('Metric','TS','Weekday','Series/Film','Length_minutes', 'Title', 'Daypart Start/End',
                           Months[4]+'(Performance) Vs '+ Qn +'(Daypart avg)',
                           Months[4]+'(Performance) Vs '+ Months[3] +'(Daypart avg)',
                           Months[4]+'(Performance) Vs '+ Months[4] +'(Daypart avg)',
                           '(Minutes per Minute) or (Session Duration)')]
else:
            Schedule_Data = Schedule_Data.loc[:,('Metric','TS','Weekday','Series/Film','Length_minutes', 'Title', 'Daypart Start/End',
                           Months[5]+'(Performance) Vs '+ Qn +'(Daypart avg)',
                           Months[5]+'(Performance) Vs '+ Months[4] +'(Daypart avg)',
                           Months[5]+'(Performance) Vs '+ Months[5] +'(Daypart avg)',
                           '(Minutes per Minute) or (Session Duration)')]

# Minutes Summary/ Session Duration Summary

In [112]:
if QuarterN == 1:
    #Minutes
    dfminsum1= dfmin.loc[:,('Series/Film','Title', 'Dayparts_Start',
                            Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                            Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                            Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                           'Minutes per Minute')]

    dfminsum2= dfmin.loc[:, ('Series/Film','Title','Dayparts_End',
                             Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                             Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart',
                             Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                            'Minutes per Minute')]
    #Session Duration
    dfsessum1= dfsessD.loc[:,('Series/Film','Title', 'Dayparts_Start',
                              Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                             'Session Duration')]

    dfsessum2= dfsessD.loc[:, ('Series/Film','Title','Dayparts_End',
                               Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                               Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart',
                               Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                              'Session Duration')]
elif QuarterN == 2:
    #Minutes
    dfminsum1= dfmin.loc[:,('Series/Film','Title', 'Dayparts_Start',
                            Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                            Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                            Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                           'Minutes per Minute')]

    dfminsum2= dfmin.loc[:, ('Series/Film','Title','Dayparts_End',
                             Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                             Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                             Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                            'Minutes per Minute')]
    #Session Duration
    dfsessum1= dfsessD.loc[:,('Series/Film','Title', 'Dayparts_Start',
                              Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                             'Session Duration')]

    dfsessum2= dfsessD.loc[:, ('Series/Film','Title','Dayparts_End',
                               Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                               Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart',
                               Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                              'Session Duration')]
else:
    #Minutes
    dfminsum1= dfmin.loc[:,('Series/Film','Title', 'Dayparts_Start',
                            Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                            Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                            Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)',
                           'Minutes per Minute')]

    dfminsum2= dfmin.loc[:, ('Series/Film','Title','Dayparts_End',
                             Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                             Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                             Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart',
                            'Minutes per Minute')]
    #Session Duration
    dfsessum1= dfsessD.loc[:,('Series/Film','Title', 'Dayparts_Start',
                              Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)',
                             'Session Duration')]

    dfsessum2= dfsessD.loc[:, ('Series/Film','Title','Dayparts_End',
                               Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart',
                               Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart',
                               Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart',
                              'Session Duration')]

In [113]:
if QuarterN == 1:
    dfminsum1.rename(columns={'Dayparts_Start':'Daypart'}, inplace=True)
    dfminsum2.rename(columns={'Dayparts_End':'Daypart',
                              Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart': Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart': Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart': Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)'
                             }, inplace=True)

    dfsessum1.rename(columns={'Dayparts_Start':'Daypart'}, inplace=True)
    dfsessum2.rename(columns={'Dayparts_End':'Daypart',
                              Months[3]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart': Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg) If crosses Daypart': Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart': Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)'
                             }, inplace=True)
elif QuarterN == 2:
    dfminsum1.rename(columns={'Dayparts_Start':'Daypart'}, inplace=True)
    dfminsum2.rename(columns={'Dayparts_End':'Daypart',
                              Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart': Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart': Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart': Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)'
                             }, inplace=True)

    dfsessum1.rename(columns={'Dayparts_Start':'Daypart'}, inplace=True)
    dfsessum2.rename(columns={'Dayparts_End':'Daypart',
                              Months[4]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart': Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg) If crosses Daypart': Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart': Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)'
                             }, inplace=True)
else:
    dfminsum1.rename(columns={'Dayparts_Start':'Daypart'}, inplace=True)
    dfminsum2.rename(columns={'Dayparts_End':'Daypart',
                              Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart': Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart': Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart': Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)'
                             }, inplace=True)

    dfsessum1.rename(columns={'Dayparts_Start':'Daypart'}, inplace=True)
    dfsessum2.rename(columns={'Dayparts_End':'Daypart',
                              Months[5]+'(Performance) vs '+ Qn +'(Daypart avg) If crosses Daypart': Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg) If crosses Daypart': Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg) If crosses Daypart': Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)'
                             }, inplace=True)

In [114]:
dfminsum= dfminsum1.append(dfminsum2)
dfsessum= dfsessum1.append(dfsessum2)

In [115]:
dfminsum= dfminsum.groupby(['Title','Daypart','Series/Film']).mean().reset_index()
dfsessum= dfsessum.groupby(['Title','Daypart','Series/Film']).mean().reset_index()

In [116]:
dfsessum.dropna(inplace=True)

# Add Difference in Daypart to min summary

In [117]:
# minsum= dfminsum.loc[:,('Title', Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)')]
# minsum.dropna(inplace=True)
# minsum= minsum.groupby('Title').mean()
# minsum.rename(columns={Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)':'OVR NOV AVG'}, inplace=True)

# dfminsum= pd.merge(dfminsum, minsum , on='Title', how='left')
# dfminsum['Diff Daypart Avg '+ Months[3]]= dfminsum[ Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)']- dfminsum['OVR NOV AVG']
# dfminsum=dfminsum.loc[:,('Title','Daypart','Series/Film',
#                          Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
#                          Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
#                          Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
#                          'Diff Daypart Avg '+ Months[3])]
# dfminsum.dropna(inplace=True)

# Schedule Data Merging Summaries

In [118]:
Dfminsum=dfminsum.rename(columns={'Minutes per Minute':'(Minutes per Minute) or (Session Duration)'})
Dfsessum=dfsessum.rename(columns={'Session Duration':'(Minutes per Minute) or (Session Duration)'})

Dfminsum['Metric']='Minutes Viewed'
Dfsessum['Metric']='Session Duration'

Min_Sess_Summ = pd.concat([Dfminsum,Dfsessum])

if QuarterN == 1:
    Min_Sess_Summ = Min_Sess_Summ.loc[:,('Metric','Title','Daypart','Series/Film',
                              Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[2] +'(Daypart avg)',
                              Months[3]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                                        '(Minutes per Minute) or (Session Duration)')]
elif QuarterN == 2:
    Min_Sess_Summ = Min_Sess_Summ.loc[:,('Metric','Title','Daypart','Series/Film',
                              Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[3] +'(Daypart avg)',
                              Months[4]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                                        '(Minutes per Minute) or (Session Duration)')]
else:
    Min_Sess_Summ = Min_Sess_Summ.loc[:,('Metric','Title','Daypart','Series/Film',
                              Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[4] +'(Daypart avg)',
                              Months[5]+'(Performance) vs '+ Months[5] +'(Daypart avg)',
                                        '(Minutes per Minute) or (Session Duration)')]

In [119]:
if QuarterN == 1:
    Min_Sess_Summ= Min_Sess_Summ[Min_Sess_Summ[Months[3]+'(Performance) vs '+ Qn +'(Daypart avg)'] .notna()]
elif QuarterN == 2:
    Min_Sess_Summ= Min_Sess_Summ[Min_Sess_Summ[Months[4]+'(Performance) vs '+ Qn +'(Daypart avg)'] .notna()]
else:
    Min_Sess_Summ= Min_Sess_Summ[Min_Sess_Summ[Months[5]+'(Performance) vs '+ Qn +'(Daypart avg)'] .notna()]

# Master/DF hourly Breakdown

In [120]:
Master['First_Hour_Min']=Master['First']*Master['Min_Hour_start']
Master['Second_Hour_Min']=Master['Mid_1']*Master['Min_Hour_Mid1']
Master['Third_Hour_Min']=Master['Mid_2']*Master['Min_Hour_Mid2']
Master['Fourth_Hour_Min']=Master['Mid_3']*Master['Min_Hour_Mid3']
Master['Last_Hour_Min']=Master['Last']*Master['Min_Hour_End']

Master['First_Hour_Sess']=Master['First']*Master['Sess_Hour_start']
Master['Second_Hour_Sess']=Master['Mid_1']*Master['Sess_Hour_Mid1']
Master['Third_Hour_Sess']=Master['Mid_2']*Master['Sess_Hour_Mid2']
Master['Fourth_Hour_Sess']=Master['Mid_3']*Master['Sess_Hour_Mid3']
Master['Last_Hour_Sess']=Master['Last']*Master['Sess_Hour_End']


In [121]:
Master['Length_1']=Master['First']*60
Master['Length_2']=Master['Mid_1']*60
Master['Length_3']=Master['Mid_2']*60
Master['Length_4']=Master['Mid_3']*60
Master['Length_5']=Master['Last']*60

In [122]:
Master1= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Hour_Start','First_Hour_Min','First_Hour_Sess',
                       'Length_1','Total Minutes','Total Session')]
Master2= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_1_Hour','Second_Hour_Min','Second_Hour_Sess',
                       'Length_2','Total Minutes','Total Session')]
Master3= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_2_Hour','Third_Hour_Min','Third_Hour_Sess',
                       'Length_3','Total Minutes','Total Session')]
Master4= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_3_Hour','Fourth_Hour_Min','Fourth_Hour_Sess',
                       'Length_4','Total Minutes','Total Session')]
Master5= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Hour_End','Last_Hour_Min','Last_Hour_Sess',
                       'Length_5','Total Minutes','Total Session')]

In [123]:
Master1.rename(columns={'Hour_Start':'Hour','First_Hour_Min':'Minutes Viewed in Hour','Length_1':'Length(min) in the Hour',
                        'First_Hour_Sess':'Sessions in Hour'},inplace=True)
Master2.rename(columns={'Middle_1_Hour':'Hour','Second_Hour_Min':'Minutes Viewed in Hour','Length_2':'Length(min) in the Hour',
                        'Second_Hour_Sess':'Sessions in Hour'},inplace=True)
Master3.rename(columns={'Middle_2_Hour':'Hour','Third_Hour_Min':'Minutes Viewed in Hour','Length_3':'Length(min) in the Hour',
                        'Third_Hour_Sess':'Sessions in Hour'},inplace=True)
Master4.rename(columns={'Middle_3_Hour':'Hour','Fourth_Hour_Min':'Minutes Viewed in Hour','Length_4':'Length(min) in the Hour',
                        'Fourth_Hour_Sess':'Sessions in Hour'},inplace=True)
Master5.rename(columns={'Hour_End':'Hour','Last_Hour_Min':'Minutes Viewed in Hour','Length_5':'Length(min) in the Hour',
                        'Last_Hour_Sess':'Sessions in Hour'},inplace=True)



In [124]:
DF= Master1.append([Master2,Master3,Master4,Master5])
DF.dropna(inplace=True)
DF['Weekday']= DF['TS'].dt.weekday

In [125]:
if Timezone == 'West':
    Conditions1 =[
        (DF['Weekday'] < 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 3),
        (DF['Weekday'] < 5) & (DF['Hour'] < 13) & (DF['Hour'] >= 6),
        (DF['Weekday'] < 5) & (DF['Hour'] < 17) & (DF['Hour'] >= 13),
        (DF['Weekday'] < 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 17),
        (DF['Weekday'] < 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] < 5) & ((DF['Hour'] < 3) | (DF['Hour'] >= 23)),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 7) & (DF['Hour'] >= 3),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 17) & (DF['Hour'] >= 7),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 17),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] >= 5) & ((DF['Hour'] < 3) | (DF['Hour'] >= 23)),

    ]
else:
    Conditions1 =[
        (DF['Weekday'] < 5) & (DF['Hour'] < 9) & (DF['Hour'] >= 6),
        (DF['Weekday'] < 5) & (DF['Hour'] < 16) & (DF['Hour'] >= 9),
        (DF['Weekday'] < 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 16),
        (DF['Weekday'] < 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] < 5) & ((DF['Hour'] < 2) | (DF['Hour'] >= 23)),
        (DF['Weekday'] < 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 2),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 10) & (DF['Hour'] >= 6),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 10),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] >= 5) & ((DF['Hour'] < 2) | (DF['Hour'] >= 23)),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 2),

    ]
    

Daypart1 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

DF['Daypart']= np.select(Conditions1, Daypart1, default= np.nan)

In [126]:
DF['Minutes per Minute']= DF['Minutes Viewed in Hour']/DF['Length(min) in the Hour']
DF['Sessions per Minute']= DF['Sessions in Hour']/DF['Length(min) in the Hour']
DF['Session Duration']= DF['Minutes Viewed in Hour']/DF['Sessions in Hour']
DF['Day']= DF['TS'].dt.day

In [127]:
Condwk=[
    DF['Weekday']== 0,DF['Weekday']== 1,DF['Weekday']== 2,DF['Weekday']== 3,
    DF['Weekday']== 4,DF['Weekday']== 5,DF['Weekday']== 6   
]

wkdays= ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

DF['Weekday(name)']= np.select(Condwk,wkdays, default=np.nan)

In [128]:
DF.dropna(inplace=True)
DF.sort_values(by=['TS','TS End'])
DF['Hour']=DF['Hour'].astype(int)
DF.rename(columns={'Hour':'Hour(24-Pac)','Weekday':'Weekday(num)','Weekday(name)':'Weekday'},inplace=True)

In [129]:
Con4=[
    (DF['Hour(24-Pac)']== 0),(DF['Hour(24-Pac)']== 1),(DF['Hour(24-Pac)']== 2),(DF['Hour(24-Pac)']== 3),
    (DF['Hour(24-Pac)']== 4),(DF['Hour(24-Pac)']== 5),(DF['Hour(24-Pac)']== 6),(DF['Hour(24-Pac)']== 7),
    (DF['Hour(24-Pac)']== 8),(DF['Hour(24-Pac)']== 9),(DF['Hour(24-Pac)']== 10),(DF['Hour(24-Pac)']== 11),
    (DF['Hour(24-Pac)']== 12),(DF['Hour(24-Pac)']== 13),(DF['Hour(24-Pac)']== 14),(DF['Hour(24-Pac)']== 15),
    (DF['Hour(24-Pac)']== 16),(DF['Hour(24-Pac)']== 17),(DF['Hour(24-Pac)']== 18),(DF['Hour(24-Pac)']== 19),
    (DF['Hour(24-Pac)']== 20),(DF['Hour(24-Pac)']== 21),(DF['Hour(24-Pac)']== 22),(DF['Hour(24-Pac)']== 23),
    
]

if Timezone == 'West':

    pac=['0am-pt','1am-pt','2am-pt','3am-pt','4am-pt','5am-pt',
        '6am-pt','7am-pt','8am-pt','9am-pt','10am-pt','11am-pt',
        '12pm-pt','1pm-pt','2pm-pt','3pm-pt','4pm-pt','5pm-pt',
        '6pm-pt','7pm-pt','8pm-pt','9pm-pt','10pm-pt','11pm-pt',]

    east=['3am-et','4am-et','5am-et','6am-et','7am-et','8am-et',
          '9am-et','10am-et','11am-et','12pm-et','1pm-et','2pm-et',
          '3pm-et','4pm-et','5pm-et','6pm-et','7pm-et','8pm-et',
          '9pm-et','10pm-et','11pm-et','0am-et','1am-et','2am-et']
else:
    pac=['9pm-pt','10pm-pt','11pm-pt','0am-pt','1am-pt','2am-pt',
         '3am-pt','4am-pt','5am-pt','6am-pt','7am-pt','8am-pt',
         '9am-pt','10am-pt','11am-pt','12pm-pt','1pm-pt','2pm-pt',
         '3pm-pt','4pm-pt','5pm-pt','6pm-pt','7pm-pt','8pm-pt']

    east=['0am-et','1am-et','2am-et','3am-et','4am-et','5am-et',
          '6am-et','7am-et','8am-et','9am-et','10am-et','11am-et',
          '12pm-et','1pm-et','2pm-et','3pm-et','4pm-et','5pm-et',
          '6pm-et','7pm-et','8pm-et','9pm-et','10pm-et','11pm-et']


DF['Pacific_Time']=np.select(Con4,pac,default=np.nan)
DF['East_Time']=np.select(Con4,east,default=np.nan)


In [130]:
DF= DF.loc[:,('Content ID','TS','TS End','Length(min) in the Hour','Day','Weekday(num)','Weekday','Hour(24-Pac)','Pacific_Time',
              'East_Time','Series/Film','Title','Daypart','Minutes per Minute','Sessions per Minute','Session Duration')]

In [131]:
DF

,Content ID,TS,TS End,Length(min) in the Hour,Day,Weekday(num),Weekday,Hour(24-Pac),Pacific_Time,East_Time,Series/Film,Title,Daypart,Minutes per Minute,Sessions per Minute,Session Duration
7,1000000002865,2022-02-01 00:02:01.030,2022-02-01 00:39:15.200,37.236167,1,1,Tue,0,9pm-pt,0am-et,Series,Season 11_Episode 3,Late Night,121.149667,39.883333,3.037601
14,1000000002864,2022-02-01 00:39:15.200,2022-02-01 01:16:47.130,21.000000,1,1,Tue,0,9pm-pt,0am-et,Series,Season 11_Episode 4,Late Night,121.149667,39.883333,3.037601
21,1000000002863,2022-02-01 01:16:47.130,2022-02-01 01:57:11.010,40.398000,1,1,Tue,1,10pm-pt,1am-et,Series,Season 11_Episode 5,Late Night,116.905000,28.616667,4.085207
29,1000000002862,2022-02-01 01:57:11.010,2022-02-01 02:36:38.120,3.000000,1,1,Tue,1,10pm-pt,1am-et,Series,Season 11_Episode 6,Late Night,116.905000,28.616667,4.085207
36,1000000002861,2022-02-01 02:36:38.120,2022-02-01 03:14:41.070,24.000000,1,1,Tue,2,11pm-pt,2am-et,Series,Season 11_Episode 7,Overnight,105.073833,20.950000,5.015457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7613,1000000000085,2022-02-28 18:46:25.200,2022-02-28 19:24:31.000,24.000000,28,0,Mon,19,4pm-pt,7pm-et,Series,Season 8_Episode 4,Weekday Fringe,79.332500,51.216667,1.548959
7621,1000000000082,2022-02-28 19:24:31.000,2022-02-28 20:04:50.260,4.000000,28,0,Mon,20,5pm-pt,8pm-et,Series,Season 8_Episode 5,Prime,91.811167,57.700000,1.591181
7637,1000000000087,2022-02-28 20:44:59.070,2022-02-28 21:23:37.260,23.000000,28,0,Mon,21,6pm-pt,9pm-et,Series,Season 8_Episode 7,Prime,89.817167,58.233333,1.542367
7644,1000000000089,2022-02-28 21:23:37.260,2022-02-28 22:02:07.260,2.000000,28,0,Mon,22,7pm-pt,10pm-et,Series,Season 8_Episode 8,Prime,92.283500,57.316667,1.610064


# Building Confidence Interval

In [132]:
DF.sort_values(by=['TS','Hour(24-Pac)'], inplace=True)
DF=DF.reset_index()

In [133]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Day'] = DF['TS End'].dt.day

In [134]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Weekday(num)'] = DF['TS End'].dt.weekday

In [135]:
Condwk=[
    DF['Weekday(num)']== 0,DF['Weekday(num)']== 1,DF['Weekday(num)']== 2,DF['Weekday(num)']== 3,
    DF['Weekday(num)']== 4,DF['Weekday(num)']== 5,DF['Weekday(num)']== 6   
]

wkdays= ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

DF['Weekday(name)']= np.select(Condwk,wkdays, default=np.nan)

In [136]:
if Timezone == 'West':
    Conditions1 =[
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 3),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 13) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 17) & (DF['Hour(24-Pac)'] >= 13),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 17),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] < 5) & ((DF['Hour(24-Pac)'] < 3) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 7) & (DF['Hour(24-Pac)'] >= 3),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 17) & (DF['Hour(24-Pac)'] >= 7),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 17),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] >= 5) & ((DF['Hour(24-Pac)'] < 3) | (DF['Hour(24-Pac)'] >= 23)),

    ]
else:
    Conditions1 =[
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 9) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 16) & (DF['Hour(24-Pac)'] >= 9),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 16),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] < 5) & ((DF['Hour(24-Pac)'] < 2) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 2),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 10) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 10),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] >= 5) & ((DF['Hour(24-Pac)'] < 2) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 2),

    ]
    

Daypart1 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

DF['Daypart1']= np.select(Conditions1, Daypart1, default= np.nan)

In [137]:
DF.rename(columns={'Daypart':'no','Daypart1':'Daypart'}, inplace=True)

In [138]:
Cond=[
    (DF['Weekday(num)']== 5) | (DF['Weekday(num)']==6),
    (DF['Weekday(num)']== 0)|(DF['Weekday(num)']==1)|(DF['Weekday(num)']== 2)|(DF['Weekday(num)']==3)|
    (DF['Weekday(num)']== 4)
]

Values=['Weekend','Weekday']

DF['Weekdays/Weekends']= np.select(Cond,Values)

# What is an Average

In [139]:
HR=DF.groupby(['Weekdays/Weekends','Day','Hour(24-Pac)']).mean().reset_index()

In [140]:
Hourly_AVG= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)')].groupby(['Weekdays/Weekends',
                                        'Hour(24-Pac)']).mean().reset_index()
Hourly_STD= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)')].groupby(['Weekdays/Weekends',
                                        'Hour(24-Pac)']).std().reset_index()
Hourly_Count= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)')].groupby(['Weekdays/Weekends',
                                        'Hour(24-Pac)']).count().reset_index()

In [141]:
Hourly_AVG.rename(columns={'Minutes per Minute':'Hourly AVG'},inplace=True)
Hourly_STD.rename(columns={'Minutes per Minute':'Hourly STD'},inplace=True)
Hourly_Count.rename(columns={'Minutes per Minute':'Hourly Count'},inplace=True)

In [142]:
DF= pd.merge(DF,Hourly_AVG, on=['Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,Hourly_STD, on=['Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,Hourly_Count, on=['Weekdays/Weekends','Hour(24-Pac)'], how='left')

# Adjust T-Value

In [143]:
data={'Count':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30],
      'Tvalue':[12.706,4.303,3.182,2.776,2.571,2.447,2.365,2.306,2.262,2.228,2.201,2.179,2.16,2.145,2.131,2.12,2.11,2.101,2.093,2.086,2.08,2.074,2.069,2.064,2.06,2.056,2.052,2.048,2.045,2.042]   
}
T_values=pd.DataFrame(data)

In [144]:
DF=pd.merge(DF,T_values, left_on='Hourly Count',right_on='Count', how='left')

In [145]:
DF['Upper CI(95%)']= DF['Hourly AVG']+(DF['Tvalue']* (DF['Hourly STD']/(DF['Hourly Count']**(1/2))))
DF['Lower CI(95%)']= DF['Hourly AVG']-(DF['Tvalue']* (DF['Hourly STD']/(DF['Hourly Count']**(1/2))))

In [146]:
Con1=[
    (DF['Minutes per Minute'] > DF['Upper CI(95%)']),
    (DF['Minutes per Minute'] < DF['Lower CI(95%)']),
    (DF['Minutes per Minute'] <= DF['Upper CI(95%)']) & (DF['Minutes per Minute'] >= DF['Lower CI(95%)'])     
]

Values1= ['Above','Below','Neutral']

DF['Above/Below/Neutral']=np.select(Con1,Values1, default=np.nan)

In [147]:
Con1=[
    (DF['Above/Below/Neutral']== 'Above'), (DF['Above/Below/Neutral']== 'Below'), (DF['Above/Below/Neutral']== 'Neutral'),
]

Values1=[(DF['Minutes per Minute']-DF['Upper CI(95%)']),
         (DF['Minutes per Minute']-DF['Lower CI(95%)']),
         0]
DF['Minutes Diff CI']= np.select(Con1,Values1, default=np.nan)

In [148]:
DF['%Minutes Diff CI']=(DF['Minutes Diff CI']/DF['Hourly AVG'])

In [149]:
#DF=pd.merge(DF,glossary_content.loc[:,('Content ID','Type')], on='Content ID', how='left')
#DF=pd.merge(DF,glossary_content.loc[:,('Content ID')], on='Content ID', how='left')

In [150]:
DF['Minutes Diff']= DF['Minutes per Minute']-DF['Hourly AVG']
DF['%Minutes Diff']= DF['Minutes Diff']/DF['Hourly AVG']
DF['Total Minutes in Hour']= DF['Minutes per Minute']*DF['Length(min) in the Hour']
DF['Total Sessions in Hour']= DF['Sessions per Minute']*DF['Length(min) in the Hour']

# Adding 3 Month CI Avg

In [151]:
M3= Amagi_DP.loc[(Amagi_DP['Month']== m2) | (Amagi_DP['Month']== m3) | (Amagi_DP['Month']== m4),('Month','Hour','week/weekend','Total Viewership Minutes per minute')]

In [152]:
M3AVG= M3.groupby(['week/weekend','Hour']).mean().reset_index()
M3STD= M3.groupby(['week/weekend','Hour']).std().reset_index()
M3Count= M3.groupby(['week/weekend','Hour']).count().reset_index()

In [153]:
M3AVG.rename(columns={'Total Viewership Minutes per minute':'M3Hourly AVG','week/weekend':'Weekdays/Weekends','Hour':'Hour(24-Pac)'},inplace=True)
M3STD.rename(columns={'Total Viewership Minutes per minute':'M3Hourly STD','week/weekend':'Weekdays/Weekends','Hour':'Hour(24-Pac)'},inplace=True)
M3Count.rename(columns={'Total Viewership Minutes per minute':'M3Hourly Count','week/weekend':'Weekdays/Weekends','Hour':'Hour(24-Pac)'},inplace=True)

In [154]:
DF= pd.merge(DF,M3AVG, on=['Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,M3STD, on=['Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,M3Count, on=['Weekdays/Weekends','Hour(24-Pac)'], how='left')

In [155]:
DF['Upper CI(95%) (3M)']= DF['M3Hourly AVG']+(1.96* (DF['M3Hourly STD']/(DF['M3Hourly Count']**(1/2))))
DF['Lower CI(95%) (3M)']= DF['M3Hourly AVG']-(1.96* (DF['M3Hourly STD']/(DF['M3Hourly Count']**(1/2))))

In [156]:
Con1=[
    (DF['Minutes per Minute'] > DF['Upper CI(95%) (3M)']),
    (DF['Minutes per Minute'] < DF['Lower CI(95%) (3M)']),
    (DF['Minutes per Minute'] <= DF['Upper CI(95%) (3M)']) & (DF['Minutes per Minute'] >= DF['Lower CI(95%) (3M)'])     
]

Values1= ['Above(3M)','Below(3M)','Neutral(3M)']

DF['Above/Below/Neutral(3M)']=np.select(Con1,Values1, default=np.nan)

In [157]:
Con1=[
    (DF['Above/Below/Neutral(3M)']== 'Above(3M)'), (DF['Above/Below/Neutral(3M)']== 'Below(3M)'), (DF['Above/Below/Neutral(3M)']== 'Neutral(3M)'),
]

Values1=[(DF['Minutes per Minute']-DF['Upper CI(95%) (3M)']),
         (DF['Minutes per Minute']-DF['Lower CI(95%) (3M)']),
         0]
DF['Minutes Diff CI (3M)']= np.select(Con1,Values1, default=np.nan)

In [158]:
DF['%Minutes Diff CI (3M)']=(DF['Minutes Diff CI (3M)']/DF['M3Hourly AVG'])
DF['Minutes Diff (3M)']= DF['Minutes per Minute']-DF['M3Hourly AVG']
DF['%Minutes Diff (3M)']= DF['Minutes Diff (3M)']/DF['M3Hourly AVG']

In [159]:
DF.sort_values(by=['TS','Hour(24-Pac)'], inplace=True)
DF.drop_duplicates(inplace=True)

In [160]:
DF=DF.loc[:,('Content ID','TS','TS End','Day','Weekday(name)','Hour(24-Pac)','Pacific_Time','East_Time','Weekdays/Weekends',
             'Daypart','Series/Film','Title','Length(min) in the Hour','Total Minutes in Hour','Total Sessions in Hour','Minutes per Minute','Sessions per Minute',
            'Session Duration','Minutes Diff','%Minutes Diff','Hourly AVG','Hourly STD','Upper CI(95%)','Lower CI(95%)','Minutes Diff CI','%Minutes Diff CI','%Minutes Diff CI (3M)','Above/Below/Neutral')]

In [161]:
DF.head(200)

,Content ID,TS,TS End,Day,Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Daypart,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,%Minutes Diff CI (3M),Above/Below/Neutral
0,1000000002865,2022-02-01 00:02:01.030,2022-02-01 00:39:15.200,1,Tue,0,9pm-pt,0am-et,Weekday,Late Night,Series,Season 11_Episode 3,37.236167,4511.149180,1485.102447,121.149667,39.883333,3.037601,31.503258,0.351417,89.646409,11.083520,94.816248,84.476570,26.333419,0.293748,0.374970,Above
1,1000000002864,2022-02-01 00:39:15.200,2022-02-01 01:16:47.130,1,Tue,0,9pm-pt,0am-et,Weekday,Late Night,Series,Season 11_Episode 4,21.000000,2544.143000,837.550000,121.149667,39.883333,3.037601,31.503258,0.351417,89.646409,11.083520,94.816248,84.476570,26.333419,0.293748,0.374970,Above
2,1000000002864,2022-02-01 00:39:15.200,2022-02-01 01:16:47.130,1,Tue,1,10pm-pt,1am-et,Weekday,Late Night,Series,Season 11_Episode 4,16.000000,1870.480000,457.866667,116.905000,28.616667,4.085207,33.324025,0.398703,83.580975,12.306818,89.321414,77.840536,27.583586,0.330022,0.437290,Above
3,1000000002863,2022-02-01 01:16:47.130,2022-02-01 01:57:11.010,1,Tue,1,10pm-pt,1am-et,Weekday,Late Night,Series,Season 11_Episode 5,40.398000,4722.728190,1156.056100,116.905000,28.616667,4.085207,33.324025,0.398703,83.580975,12.306818,89.321414,77.840536,27.583586,0.330022,0.437290,Above
4,1000000002862,2022-02-01 01:57:11.010,2022-02-01 02:36:38.120,1,Tue,1,10pm-pt,1am-et,Weekday,Late Night,Series,Season 11_Episode 6,3.000000,350.715000,85.850000,116.905000,28.616667,4.085207,33.324025,0.398703,83.580975,12.306818,89.321414,77.840536,27.583586,0.330022,0.437290,Above
5,1000000002862,2022-02-01 01:57:11.010,2022-02-01 02:36:38.120,1,Tue,2,11pm-pt,2am-et,Weekday,Overnight,Series,Season 11_Episode 6,36.000000,3782.658000,754.200000,105.073833,20.950000,5.015457,24.569667,0.305197,80.504167,11.972204,86.088527,74.919807,18.985307,0.235830,0.379618,Above
6,1000000002861,2022-02-01 02:36:38.120,2022-02-01 03:14:41.070,1,Tue,2,11pm-pt,2am-et,Weekday,Overnight,Series,Season 11_Episode 7,24.000000,2521.772000,502.800000,105.073833,20.950000,5.015457,24.569667,0.305197,80.504167,11.972204,86.088527,74.919807,18.985307,0.235830,0.379618,Above
7,1000000002861,2022-02-01 02:36:38.120,2022-02-01 03:14:41.070,1,Tue,3,0am-pt,3am-et,Weekday,Overnight,Series,Season 11_Episode 7,14.000000,1461.250000,282.800000,104.375000,20.200000,5.167079,24.019367,0.298913,80.355633,11.841694,85.879118,74.832149,18.495882,0.230175,0.442194,Above
8,1000000002860,2022-02-01 03:14:41.070,2022-02-01 03:54:38.190,1,Tue,3,0am-pt,3am-et,Weekday,Overnight,Series,Season 11_Episode 8,39.952000,4169.990000,807.030400,104.375000,20.200000,5.167079,24.019367,0.298913,80.355633,11.841694,85.879118,74.832149,18.495882,0.230175,0.442194,Above
9,1000000002859,2022-02-01 03:54:38.190,2022-02-01 04:34:00.130,1,Tue,3,0am-pt,3am-et,Weekday,Overnight,Series,Season 11_Episode 9,6.000000,626.250000,121.200000,104.375000,20.200000,5.167079,24.019367,0.298913,80.355633,11.841694,85.879118,74.832149,18.495882,0.230175,0.442194,Above


# Summary no Dayparts

In [162]:
W=DF.loc[:,('Title','Series/Film','Above/Below/Neutral','Content ID')]

In [163]:
W=W.pivot_table(index= ['Series/Film','Title'], values='Content ID', columns='Above/Below/Neutral', aggfunc=pd.Series.count)
W=W.reset_index()

In [164]:
W['Above']=W['Above'].fillna(0)
W['Below']=W['Below'].fillna(0)
W['Neutral']=W['Neutral'].fillna(0)


W['Count']= (W['Above']+ W['Below']+ W['Neutral'])
W['%Below CI']= W['Below']/W['Count']
W['%Inside CI']= W['Neutral']/W['Count']
W['%Above CI']= W['Above']/W['Count']
W['Score']= ((W['Above']**2)-(W['Below']**2))/W['Count']

In [165]:
W1=DF.loc[:,('Title','Series/Film','Minutes Diff CI','%Minutes Diff CI','Minutes Diff','%Minutes Diff','Minutes per Minute','%Minutes Diff CI (3M)')]
W1= W1.groupby('Title').mean().reset_index()

In [166]:
SummaryW= pd.merge(W,W1, on='Title', how='left')

In [167]:
#Combined Score

CondSC=[
    (SummaryW['Score']> 0) & (SummaryW['%Minutes Diff CI']> 0),
     (SummaryW['Score']> 0) & (SummaryW['%Minutes Diff CI']< 0),
     (SummaryW['Score']== 0) | (SummaryW['%Minutes Diff CI']== 0),
     (SummaryW['Score']< 0) & (SummaryW['%Minutes Diff CI']< 0)   
]

ValuesSC=[SummaryW['Score']*SummaryW['%Minutes Diff CI'],
         0,
         0,
         (SummaryW['Score']*SummaryW['%Minutes Diff CI'])*-1
        ]

SummaryW['Combined Score']= np.select(CondSC,ValuesSC)

In [168]:
Summary_no_Dayparts= SummaryW.loc[:,('Series/Film','Title','Count','Above','Neutral','Below','%Above CI','%Inside CI',
                         '%Below CI','Score','Minutes per Minute','%Minutes Diff CI','%Minutes Diff CI (3M)','Combined Score')]

In [169]:
Summary_no_Dayparts

,Series/Film,Title,Count,Above,Neutral,Below,%Above CI,%Inside CI,%Below CI,Score,Minutes per Minute,%Minutes Diff CI,%Minutes Diff CI (3M),Combined Score
0,Series,Season 10_Episode 10,15.0,6.0,7.0,2.0,0.400000,0.466667,0.133333,2.133333,87.304833,0.043926,0.104313,0.093708
1,Series,Season 10_Episode 11,8.0,5.0,3.0,0.0,0.625000,0.375000,0.000000,3.125000,90.459583,0.105225,0.207911,0.328827
2,Series,Season 10_Episode 12,11.0,5.0,5.0,1.0,0.454545,0.454545,0.090909,2.181818,88.970076,0.072293,0.142383,0.157729
3,Series,Season 10_Episode 3,9.0,4.0,3.0,2.0,0.444444,0.333333,0.222222,1.333333,88.853537,0.025807,0.106108,0.034410
4,Series,Season 10_Episode 4,11.0,6.0,2.0,3.0,0.545455,0.181818,0.272727,2.454545,88.033470,0.030214,0.121468,0.074162
5,Series,Season 10_Episode 5,9.0,5.0,3.0,1.0,0.555556,0.333333,0.111111,2.666667,88.507852,0.033875,0.123760,0.090334
6,Series,Season 10_Episode 6,11.0,7.0,2.0,2.0,0.636364,0.181818,0.181818,4.090909,86.639288,0.024324,0.131288,0.099507
7,Series,Season 10_Episode 7,9.0,6.0,2.0,1.0,0.666667,0.222222,0.111111,3.888889,91.373500,0.054973,0.169356,0.213784
8,Series,Season 10_Episode 8,12.0,6.0,3.0,3.0,0.500000,0.250000,0.250000,2.250000,86.885431,0.048022,0.110583,0.108050
9,Series,Season 10_Episode 9,8.0,4.0,4.0,0.0,0.500000,0.500000,0.000000,2.000000,90.531729,0.060557,0.170869,0.121114


# Summary With Dayparts

In [170]:
CSD=DF.loc[:,('Title','Daypart','Series/Film','Above/Below/Neutral','Content ID')]

In [171]:
CSD=CSD.pivot_table(index= ['Series/Film','Title','Daypart'], values='Content ID', columns='Above/Below/Neutral', aggfunc=pd.Series.count)
CSD=CSD.reset_index()

In [172]:
CSD['Above']=CSD['Above'].fillna(0)
CSD['Below']=CSD['Below'].fillna(0)
CSD['Neutral']=CSD['Neutral'].fillna(0)


CSD['Count']= (CSD['Above']+ CSD['Below']+ CSD['Neutral'])
CSD['%Below CI']= CSD['Below']/CSD['Count']
CSD['%Inside CI']= CSD['Neutral']/CSD['Count']
CSD['%Above CI']= CSD['Above']/CSD['Count']
CSD['Score']= ((CSD['Above']**2)-(CSD['Below']**2))/CSD['Count']

In [173]:
CSD1=DF.loc[:,('Title','Series/Film','Daypart','Minutes Diff CI','Minutes per Minute','%Minutes Diff CI','Minutes Diff','%Minutes Diff','%Minutes Diff CI (3M)')]
CSD1= CSD1.groupby(['Title','Daypart']).mean().reset_index()
CSDsumm= pd.merge(CSD,CSD1, on=['Title','Daypart'], how='left')

In [174]:
CondSCD=[
    (CSDsumm['Score']> 0) & (CSDsumm['%Minutes Diff CI']> 0),
     (CSDsumm['Score']> 0) & (CSDsumm['%Minutes Diff CI']< 0),
     (CSDsumm['Score']== 0) | (CSDsumm['%Minutes Diff CI']== 0),
     (CSDsumm['Score']< 0) & (CSDsumm['%Minutes Diff CI']< 0)   
]

ValuesSCD=[CSDsumm['Score']*CSDsumm['%Minutes Diff CI'],
         0,
         0,
         (CSDsumm['Score']*CSDsumm['%Minutes Diff CI'])*-1
        ]

CSDsumm['Combined Score']= np.select(CondSCD,ValuesSCD)

In [175]:
# CSDsumm=pd.merge(CSDsumm,Summary_no_Dayparts.loc[:,('Title','%Minutes Diff CI','%Minutes Diff')], on='Title', how='left')

In [176]:
# CSDsumm['(%Min Diff CI Daypart) vs (%Min Diff CI All)']= CSDsumm['%Minutes Diff CI_x']- CSDsumm['%Minutes Diff CI_y']
# CSDsumm['(%Min Diff Daypart) vs (%Min Diff All)']= CSDsumm['%Minutes Diff_x']- CSDsumm['%Minutes Diff_y']

In [177]:
 CSDsumm=CSDsumm.loc[:,('Series/Film','Title','Daypart','Count','Above','Neutral','Below','%Above CI','%Inside CI',
                          '%Below CI','Score','Minutes per Minute','%Minutes Diff CI','%Minutes Diff CI (3M)','Combined Score'
                       )]

# CSDsumm.rename(columns={'%Minutes Diff CI_x':'%Minutes Diff CI','%Minutes Diff_x':'%Minutes Diff'}, inplace = True)

In [178]:
Summary_with_Dayparts=CSDsumm

In [179]:
if Timezone == 'East':
    DF.rename(columns={'Hour(24-Pac)':'Hour(24-East)'},inplace=True)
    MonthlyAVGs.rename(columns={'Hour(24-Pac)':'Hour(24-East)'},inplace=True)    

# Platforms Monthly total Minutes

In [180]:
Platform.head(200)

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Platform,Channel,Weekday,Hour,Day,Month,Daypart
58,2021-10-01 00:00:00,Towie Roku US,2193,3028.52,1764,Roku,Towie,4,0,1,10,Late Night
119,2021-10-01 01:00:00,Towie Roku US,1588,3421.42,1278,Roku,Towie,4,1,1,10,Late Night
175,2021-10-01 02:00:00,Towie Roku US,1294,3559.75,1029,Roku,Towie,4,2,1,10,Overnight
238,2021-10-01 03:00:00,Towie Roku US,1109,3228.58,883,Roku,Towie,4,3,1,10,Overnight
297,2021-10-01 04:00:00,Towie Roku US,893,2512.98,730,Roku,Towie,4,4,1,10,Overnight
352,2021-10-01 05:00:00,Towie Roku US,846,2071.67,697,Roku,Towie,4,5,1,10,Overnight
403,2021-10-01 06:00:00,Towie Roku US,1039,2230.33,832,Roku,Towie,4,6,1,10,Weekday Morning
460,2021-10-01 07:00:00,Towie Roku US,1214,2657.73,1023,Roku,Towie,4,7,1,10,Weekday Morning
517,2021-10-01 08:00:00,Towie Roku US,1374,2786.15,1130,Roku,Towie,4,8,1,10,Weekday Morning
574,2021-10-01 09:00:00,Towie Roku US,1583,2996.98,1303,Roku,Towie,4,9,1,10,Weekday Day


In [181]:
Platform['Year']=Platform['Date'].dt.year

In [182]:
CondMonth=[
    Platform['Month'] == 1,Platform['Month'] == 2,Platform['Month'] == 3,
    Platform['Month'] == 4,Platform['Month'] == 5,Platform['Month'] == 6,
    Platform['Month'] == 7,Platform['Month'] == 8,Platform['Month'] == 9,
    Platform['Month'] == 10,Platform['Month'] == 11,Platform['Month'] == 12,
]

ValueM=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

Platform['Months']= np.select(CondMonth,ValueM)

In [183]:
Platform

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Platform,Channel,Weekday,Hour,Day,Month,Daypart,Year,Months
58,2021-10-01 00:00:00,Towie Roku US,2193,3028.52,1764,Roku,Towie,4,0,1,10,Late Night,2021,Oct
119,2021-10-01 01:00:00,Towie Roku US,1588,3421.42,1278,Roku,Towie,4,1,1,10,Late Night,2021,Oct
175,2021-10-01 02:00:00,Towie Roku US,1294,3559.75,1029,Roku,Towie,4,2,1,10,Overnight,2021,Oct
238,2021-10-01 03:00:00,Towie Roku US,1109,3228.58,883,Roku,Towie,4,3,1,10,Overnight,2021,Oct
297,2021-10-01 04:00:00,Towie Roku US,893,2512.98,730,Roku,Towie,4,4,1,10,Overnight,2021,Oct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237109,2022-02-28 21:00:00,Towie Roku US,3494,5389.03,2812,Roku,Towie,0,21,28,2,Prime,2022,Feb
237175,2022-02-28 22:00:00,TheOnlyWayIsEssex_LG,245,281.38,217,LG,Towie,0,22,28,2,Prime,2022,Feb
237176,2022-02-28 22:00:00,Towie Roku US,3194,5255.63,2568,Roku,Towie,0,22,28,2,Prime,2022,Feb
237238,2022-02-28 23:00:00,TheOnlyWayIsEssex_LG,473,682.07,446,LG,Towie,0,23,28,2,Late Night,2022,Feb


In [184]:
Platform= Platform.pivot_table(index='Platform',values='Total Viewership Minutes',columns='Months', aggfunc=pd.Series.sum).reset_index()

In [185]:
Platform

Months,Platform,Dec,Feb,Jan,Nov,Oct
0,LG,383737.47,82409.19,346966.79,112520.96,226.84
1,Roku,3255364.89,3377539.09,4063786.96,2631510.95,2363673.08


In [186]:
Platform.columns=('Platform','Dec-21','Feb-22','Jan-22','Nov-21','Oct-21')
Platform=Platform.loc[:,('Platform','Oct-21','Nov-21','Dec-21','Jan-22','Feb-22')]

In [187]:
Platform

,Platform,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22
0,LG,226.84,112520.96,383737.47,346966.79,82409.19
1,Roku,2363673.08,2631510.95,3255364.89,4063786.96,3377539.09


# All Tables Last Look

In [188]:
pd.set_option('display.max_columns',None)

In [189]:
DF.head(200)

,Content ID,TS,TS End,Day,Weekday(name),Hour(24-East),Pacific_Time,East_Time,Weekdays/Weekends,Daypart,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,%Minutes Diff CI (3M),Above/Below/Neutral
0,1000000002865,2022-02-01 00:02:01.030,2022-02-01 00:39:15.200,1,Tue,0,9pm-pt,0am-et,Weekday,Late Night,Series,Season 11_Episode 3,37.236167,4511.149180,1485.102447,121.149667,39.883333,3.037601,31.503258,0.351417,89.646409,11.083520,94.816248,84.476570,26.333419,0.293748,0.374970,Above
1,1000000002864,2022-02-01 00:39:15.200,2022-02-01 01:16:47.130,1,Tue,0,9pm-pt,0am-et,Weekday,Late Night,Series,Season 11_Episode 4,21.000000,2544.143000,837.550000,121.149667,39.883333,3.037601,31.503258,0.351417,89.646409,11.083520,94.816248,84.476570,26.333419,0.293748,0.374970,Above
2,1000000002864,2022-02-01 00:39:15.200,2022-02-01 01:16:47.130,1,Tue,1,10pm-pt,1am-et,Weekday,Late Night,Series,Season 11_Episode 4,16.000000,1870.480000,457.866667,116.905000,28.616667,4.085207,33.324025,0.398703,83.580975,12.306818,89.321414,77.840536,27.583586,0.330022,0.437290,Above
3,1000000002863,2022-02-01 01:16:47.130,2022-02-01 01:57:11.010,1,Tue,1,10pm-pt,1am-et,Weekday,Late Night,Series,Season 11_Episode 5,40.398000,4722.728190,1156.056100,116.905000,28.616667,4.085207,33.324025,0.398703,83.580975,12.306818,89.321414,77.840536,27.583586,0.330022,0.437290,Above
4,1000000002862,2022-02-01 01:57:11.010,2022-02-01 02:36:38.120,1,Tue,1,10pm-pt,1am-et,Weekday,Late Night,Series,Season 11_Episode 6,3.000000,350.715000,85.850000,116.905000,28.616667,4.085207,33.324025,0.398703,83.580975,12.306818,89.321414,77.840536,27.583586,0.330022,0.437290,Above
5,1000000002862,2022-02-01 01:57:11.010,2022-02-01 02:36:38.120,1,Tue,2,11pm-pt,2am-et,Weekday,Overnight,Series,Season 11_Episode 6,36.000000,3782.658000,754.200000,105.073833,20.950000,5.015457,24.569667,0.305197,80.504167,11.972204,86.088527,74.919807,18.985307,0.235830,0.379618,Above
6,1000000002861,2022-02-01 02:36:38.120,2022-02-01 03:14:41.070,1,Tue,2,11pm-pt,2am-et,Weekday,Overnight,Series,Season 11_Episode 7,24.000000,2521.772000,502.800000,105.073833,20.950000,5.015457,24.569667,0.305197,80.504167,11.972204,86.088527,74.919807,18.985307,0.235830,0.379618,Above
7,1000000002861,2022-02-01 02:36:38.120,2022-02-01 03:14:41.070,1,Tue,3,0am-pt,3am-et,Weekday,Overnight,Series,Season 11_Episode 7,14.000000,1461.250000,282.800000,104.375000,20.200000,5.167079,24.019367,0.298913,80.355633,11.841694,85.879118,74.832149,18.495882,0.230175,0.442194,Above
8,1000000002860,2022-02-01 03:14:41.070,2022-02-01 03:54:38.190,1,Tue,3,0am-pt,3am-et,Weekday,Overnight,Series,Season 11_Episode 8,39.952000,4169.990000,807.030400,104.375000,20.200000,5.167079,24.019367,0.298913,80.355633,11.841694,85.879118,74.832149,18.495882,0.230175,0.442194,Above
9,1000000002859,2022-02-01 03:54:38.190,2022-02-01 04:34:00.130,1,Tue,3,0am-pt,3am-et,Weekday,Overnight,Series,Season 11_Episode 9,6.000000,626.250000,121.200000,104.375000,20.200000,5.167079,24.019367,0.298913,80.355633,11.841694,85.879118,74.832149,18.495882,0.230175,0.442194,Above


In [190]:
Summary_no_Dayparts

,Series/Film,Title,Count,Above,Neutral,Below,%Above CI,%Inside CI,%Below CI,Score,Minutes per Minute,%Minutes Diff CI,%Minutes Diff CI (3M),Combined Score
0,Series,Season 10_Episode 10,15.0,6.0,7.0,2.0,0.400000,0.466667,0.133333,2.133333,87.304833,0.043926,0.104313,0.093708
1,Series,Season 10_Episode 11,8.0,5.0,3.0,0.0,0.625000,0.375000,0.000000,3.125000,90.459583,0.105225,0.207911,0.328827
2,Series,Season 10_Episode 12,11.0,5.0,5.0,1.0,0.454545,0.454545,0.090909,2.181818,88.970076,0.072293,0.142383,0.157729
3,Series,Season 10_Episode 3,9.0,4.0,3.0,2.0,0.444444,0.333333,0.222222,1.333333,88.853537,0.025807,0.106108,0.034410
4,Series,Season 10_Episode 4,11.0,6.0,2.0,3.0,0.545455,0.181818,0.272727,2.454545,88.033470,0.030214,0.121468,0.074162
5,Series,Season 10_Episode 5,9.0,5.0,3.0,1.0,0.555556,0.333333,0.111111,2.666667,88.507852,0.033875,0.123760,0.090334
6,Series,Season 10_Episode 6,11.0,7.0,2.0,2.0,0.636364,0.181818,0.181818,4.090909,86.639288,0.024324,0.131288,0.099507
7,Series,Season 10_Episode 7,9.0,6.0,2.0,1.0,0.666667,0.222222,0.111111,3.888889,91.373500,0.054973,0.169356,0.213784
8,Series,Season 10_Episode 8,12.0,6.0,3.0,3.0,0.500000,0.250000,0.250000,2.250000,86.885431,0.048022,0.110583,0.108050
9,Series,Season 10_Episode 9,8.0,4.0,4.0,0.0,0.500000,0.500000,0.000000,2.000000,90.531729,0.060557,0.170869,0.121114


In [191]:
Summary_with_Dayparts

,Series/Film,Title,Daypart,Count,Above,Neutral,Below,%Above CI,%Inside CI,%Below CI,Score,Minutes per Minute,%Minutes Diff CI,%Minutes Diff CI (3M),Combined Score
0,Series,Season 10_Episode 10,Late Night,2.0,1.0,1.0,0.0,0.500000,0.500000,0.000000,0.500000,92.214333,0.003789,0.077218,0.001895
1,Series,Season 10_Episode 10,Late Night (wknd),1.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,-1.000000,73.281333,-0.117189,-0.171665,-0.117189
2,Series,Season 10_Episode 10,Overnight,1.0,1.0,0.0,0.0,1.000000,0.000000,0.000000,1.000000,90.999167,0.063717,0.250606,0.063717
3,Series,Season 10_Episode 10,Overnight (wknd),2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.000000,83.812250,0.000000,0.082949,0.000000
4,Series,Season 10_Episode 10,Prime (wknd),1.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,-1.000000,80.122333,-0.070010,-0.143863,-0.070010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,Series,Season 9_The Only Way is Marbs Part 1,Overnight,1.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,-1.000000,69.221167,-0.070787,-0.006250,-0.070787
859,Series,Season 9_The Only Way is Marbs Part 1,Prime,1.0,0.0,1.0,0.0,0.000000,1.000000,0.000000,0.000000,98.964667,0.000000,0.000000,0.000000
860,Series,Season 9_The Only Way is Marbs Part 1,Prime (wknd),3.0,1.0,1.0,1.0,0.333333,0.333333,0.333333,0.000000,97.938944,0.006072,-0.041207,0.000000
861,Series,Season 9_The Only Way is Marbs Part 1,Weekday Day,4.0,0.0,0.0,4.0,0.000000,0.000000,1.000000,-4.000000,69.962500,-0.104819,-0.062737,-0.419275


In [192]:
Schedule_Data

,Metric,TS,Weekday,Series/Film,Length_minutes,Title,Daypart Start/End,Feb(Performance) Vs Q4(Daypart avg),Feb(Performance) Vs Jan(Daypart avg),Feb(Performance) Vs Feb(Daypart avg),(Minutes per Minute) or (Session Duration)
0,Minutes Viewed,2022-02-01 00:02:01.030,Tue,Series,37.236167,Season 11_Episode 3,Late Night/Late Night,0.692079,0.177023,0.350513,121.149667
1,Minutes Viewed,2022-02-01 00:39:15.200,Tue,Series,37.532167,Season 11_Episode 4,Late Night/Late Night,0.642814,0.142754,0.311193,117.622386
2,Minutes Viewed,2022-02-01 01:16:47.130,Tue,Series,40.398000,Season 11_Episode 5,Late Night/Late Night,0.632794,0.135784,0.303196,116.905000
3,Minutes Viewed,2022-02-01 01:57:11.010,Tue,Series,39.451833,Season 11_Episode 6,Late Night/Overnight,0.885714,0.270962,0.358022,104.770112
4,Minutes Viewed,2022-02-01 02:36:38.120,Tue,Series,38.049167,Season 11_Episode 7,Overnight/Overnight,0.884108,0.269880,0.356866,104.680926
...,...,...,...,...,...,...,...,...,...,...,...
1026,Session Duration,2022-02-28 20:44:59.070,Mon,Series,38.636500,Season 8_Episode 7,Prime/Prime,0.279785,-0.084659,-0.074903,1.562285
1027,Session Duration,2022-02-28 21:23:37.260,Mon,Series,38.500000,Season 8_Episode 8,Prime/Prime,0.266270,-0.094325,-0.084672,1.545787
1028,Session Duration,2022-02-28 22:02:07.260,Mon,Series,39.531167,Season 8_Episode 9,Prime/Prime,0.318924,-0.056665,-0.046611,1.610064
1029,Session Duration,2022-02-28 22:41:39.130,Mon,Series,40.634000,Season 8_Episode 10,Prime/Late Night,0.268636,-0.079482,-0.084666,1.849650


In [193]:
Min_Sess_Summ.head(100)

,Metric,Title,Daypart,Series/Film,Feb(Performance) vs Q4(Daypart avg),Feb(Performance) vs Jan(Daypart avg),Feb(Performance) vs Feb(Daypart avg),(Minutes per Minute) or (Session Duration)
0,Minutes Viewed,Season 10_Episode 10,Late Night,Series,0.297040,-0.097769,0.035218,92.865659
1,Minutes Viewed,Season 10_Episode 10,Late Night (wknd),Series,0.037707,-0.269788,-0.193564,77.337550
2,Minutes Viewed,Season 10_Episode 10,Overnight,Series,0.637856,0.103907,0.179524,90.999167
3,Minutes Viewed,Season 10_Episode 10,Overnight (wknd),Series,0.418437,-0.085982,-0.057790,84.078121
4,Minutes Viewed,Season 10_Episode 10,Prime (wknd),Series,-0.075518,-0.354634,-0.176231,77.337550
5,Minutes Viewed,Season 10_Episode 10,Weekday Day,Series,0.469476,-0.055650,0.111087,85.289443
6,Minutes Viewed,Season 10_Episode 10,Weekday Fringe,Series,0.626672,0.085796,0.260472,113.624198
7,Minutes Viewed,Season 10_Episode 10,Weekday Morning,Series,0.637822,0.077414,0.206490,85.914557
8,Minutes Viewed,Season 10_Episode 11,Overnight,Series,0.603854,0.080990,0.155038,89.110020
9,Minutes Viewed,Season 10_Episode 11,Overnight (wknd),Series,0.410736,-0.090944,-0.062905,83.621667


In [194]:
MonthlyAVGs

,Wk/Wknd,Pacific_Time,Hour(24-East),East_Time,Daypart,Oct,Nov,Dec,Jan,Feb,Q4,Wk/Wknd,Pacific_Time,Hour(24-East),East_Time,Daypart,Oct,Nov,Dec,Jan,Feb,Q4,Wk/Wknd,Pacific_Time,Hour(24-East),East_Time,Daypart,Oct,Nov,Dec,Jan,Feb,Q4,Wk/Wknd,Pacific_Time,Hour(24-East),East_Time,Daypart,Oct,Nov,Dec,Jan,Feb,Q4
0,Weekdays,9pm-pt,0,0am-et,Late Night,3419.524286,3961.655455,5304.183043,6059.302381,5292.5940,4228.454261,Weekdays,9pm-pt,0,0am-et,Late Night,1.521682,1.808221,2.199737,2.457913,2.509607,1.843213,Weekends,9pm-pt,0,0am-et,Late Night (wknd),3499.588,4318.04750,5452.45125,6250.991,5858.14625,4423.362250,Weekends,9pm-pt,0,0am-et,Late Night (wknd),1.365010,1.693573,1.926859,2.092789,2.479146,1.661814
1,Weekdays,10pm-pt,1,1am-et,Late Night,3136.921429,3720.259545,4918.962174,5538.807143,5014.8585,3925.381049,Weekdays,10pm-pt,1,1am-et,Late Night,1.965154,2.418387,2.722375,3.048202,3.186417,2.368639,Weekends,10pm-pt,1,1am-et,Late Night (wknd),3319.772,4469.84250,5013.69375,5853.199,5817.52625,4267.769417,Weekends,10pm-pt,1,1am-et,Late Night (wknd),1.747212,2.201526,2.385421,2.555148,3.080818,2.111386
2,Weekdays,11pm-pt,2,2am-et,Overnight,2919.497143,3493.629091,4520.336087,5237.443810,4830.2500,3644.487440,Weekdays,11pm-pt,2,2am-et,Overnight,2.512342,3.040673,3.362890,3.749188,3.849943,2.971969,Weekends,11pm-pt,2,2am-et,Overnight (wknd),3163.076,3693.79625,4618.36750,5807.073,5753.38250,3825.079917,Weekends,11pm-pt,2,2am-et,Overnight (wknd),2.238166,2.736674,2.805897,3.270013,3.666211,2.593579
3,Weekdays,0am-pt,3,3am-et,Overnight,2769.506667,3308.232273,4304.071739,4985.435238,4821.3380,3460.603560,Weekdays,0am-pt,3,3am-et,Overnight,2.739106,3.174076,3.663443,4.066052,4.347453,3.192208,Weekends,0am-pt,3,3am-et,Overnight (wknd),3187.302,3515.80375,4288.05750,5677.048,5648.55625,3663.721083,Weekends,0am-pt,3,3am-et,Overnight (wknd),2.547005,3.138560,3.114069,3.753599,4.382546,2.933212
4,Weekdays,1am-pt,4,4am-et,Overnight,2427.416190,3053.517727,4086.639565,4857.845714,4528.6235,3189.191161,Weekdays,1am-pt,4,4am-et,Overnight,2.974709,3.412664,3.970632,4.576442,4.741302,3.452668,Weekends,1am-pt,4,4am-et,Overnight (wknd),2971.304,3263.03625,4012.32875,5408.546,5177.35375,3415.556333,Weekends,1am-pt,4,4am-et,Overnight (wknd),2.893783,3.214854,3.423715,4.309995,4.710424,3.177451
5,Weekdays,2am-pt,5,5am-et,Overnight,2418.010952,2932.135000,3770.153913,4703.370000,4335.5550,3040.099955,Weekdays,2am-pt,5,5am-et,Overnight,2.812309,3.160217,3.627668,4.496239,4.620066,3.200065,Weekends,2am-pt,5,5am-et,Overnight (wknd),2828.388,3203.41875,3933.27125,5184.307,4837.10375,3321.692667,Weekends,2am-pt,5,5am-et,Overnight (wknd),2.849650,2.980906,3.213076,4.149579,4.342514,3.014544
6,Weekdays,3am-pt,6,6am-et,Weekday Morning,2437.605714,2885.636818,3879.103478,4655.477143,4310.3570,3067.448670,Weekdays,3am-pt,6,6am-et,Weekday Morning,2.394018,2.694875,3.205442,3.798675,3.974591,2.764778,Weekends,3am-pt,6,6am-et,Weekend Morning,2747.645,3287.57375,3991.71625,5131.441,4795.87750,3342.311667,Weekends,3am-pt,6,6am-et,Weekend Morning,2.489915,2.631666,2.766141,3.618852,3.924059,2.629241
7,Weekdays,4am-pt,7,7am-et,Weekday Morning,2500.689048,2987.550455,3827.290435,4729.843810,4186.5655,3105.176646,Weekdays,4am-pt,7,7am-et,Weekday Morning,2.068129,2.202084,2.701036,3.294855,3.258112,2.323750,Weekends,4am-pt,7,7am-et,Weekend Morning,2802.964,3421.94750,4184.92625,5081.700,4623.96625,3469.945917,Weekends,4am-pt,7,7am-et,Weekend Morning,1.974687,2.113333,2.446484,2.982949,3.156550,2.178168
8,Weekdays,5am-pt,8,8am-et,Weekday Morning,2644.763810,3171.079091,3992.848261,4968.137143,4320.9340,3269.563720,Weekdays,5am-pt,8,8am-et,Weekday Morning,1.869795,2.009952,2.411060,2.881489,2.881108,2.096935,Weekends,5am-pt,8,8am-et,Weekend Morning,2998.393,3747.62625,4293.57625,5232.243,4865.74750,3679.865167,Weekends,5am-pt,8,8am-et,Weekend Morning,1.596178,1.773002,1.951770,2.416382,2.482491,1.773650
9,Weekdays,6am-pt,9,9am-et,Weekday Day,2727.012381,3288.861364,4216.622609,5

In [195]:
DPs

,Metric,Wk/Wknd,Daypart,Oct,Nov,Dec,Jan,Feb,Q4,Feb V Jan,Feb V Dec,Feb V Q4
0,Minutes,Weekday,Late Night,3437.597937,4027.803030,5422.261739,6175.734921,5382.382833,4295.887569,-0.128463,-0.007355,0.252915
1,Minutes,Weekday,Overnight,2633.607738,3196.878523,4170.300326,4946.023690,4628.941625,3333.595529,-0.064108,0.109978,0.388573
2,Minutes,Weekday,Prime,3793.213016,4636.191818,6206.979130,7185.865873,5964.879833,4878.794655,-0.169915,-0.039004,0.222613
3,Minutes,Weekday,Weekday Day,2934.271224,3511.410455,4580.110870,5718.977279,4860.749500,3675.264183,-0.150067,0.061273,0.322558
4,Minutes,Weekday,Weekday Fringe,3268.902619,3968.105341,5336.124022,6278.759405,5408.652000,4191.043994,-0.138580,0.013592,0.290526
5,Minutes,Weekday,Weekday Morning,2527.686190,3014.755455,3899.747391,4784.486032,4272.618833,3147.396345,-0.106985,0.095614,0.357509
0,Minutes,Weekend,Late Night (wknd),3582.498000,4455.867083,5376.563333,6354.669333,5754.022083,4471.642806,-0.094521,0.070204,0.286780
1,Minutes,Weekend,Overnight (wknd),3037.517500,3419.013750,4213.006250,5519.243500,5354.099062,3556.512500,-0.029922,0.270850,0.505435
2,Minutes,Weekend,Prime (wknd),4045.991333,4903.527500,6108.376250,7190.109333,5632.951250,5019.298361,-0.216569,-0.077832,0.122259
3,Minutes,Weekend,Weekend Day,3543.063800,4403.965250,5127.452875,6473.780600,5490.208250,4358.160642,-0.151932,0.070748,0.259754


In [196]:
#Change Date****************************************************************************
output= pd.ExcelWriter(Channel + '_' + Months[-1]+'_'+ Day +'_out.xlsx', engine='xlsxwriter')

DF.to_excel(output, sheet_name= 'DF',index=False)
Summary_no_Dayparts.to_excel(output, sheet_name= 'CI Summary',index=False)
Summary_with_Dayparts.to_excel(output, sheet_name= 'CI Dayparts',index=False)
Schedule_Data.to_excel(output, sheet_name='Schedule Data', index=False)
Min_Sess_Summ.to_excel(output, sheet_name='Min_SessD Summary', index= False)
MonthlyAVGs.to_excel(output, sheet_name='Daypart Avrgs', index=False)
DPs.to_excel(output, sheet_name='Daypart_Summ', index=False)
Platform.to_excel(output, sheet_name='Graphs', index=False)


output.save()